# Apêndice VI – Ntbk 01. Coleta dos dados e transformações preliminares.ipynb.

Este notebook tem por objetivo a coleta e transformações preliminares nos *datasets* que serão utilizadas no projeto, tais como:
- Descompactação de arquivos;
- Criação da variável padronizada, chamada “município-uf”;
- Manutenção, apenas, das informações relativas ao ano de interesse (2018);
- Renomeação de variáveis; 
- Criação, em alguns casos, de novas colunas derivadas (enriquecimento dos dados);
- Remoção de variáveis redundantes e com valores nulos ou constantes;
- Reordenação de colunas.

Está estruturado da seguinte maneira:
1. Ações Preliminares
2. Coleta das bases de dados
3. Resumo das bases de dados criadas


# 1. Ações Preliminares


## - Inicialização da plataforma

A célula a seguir inicializa a plataforma, carregando as bibliotecas que serão relevantes para o trabalho em seguida.


In [1]:
import csv
import sys
import pandas as pd


from zipfile import ZipFile


pd.set_option('max_columns',200)

## - Declaração de funções que serão utilizadas no notebook.

In [2]:
## Função utilizada para visualizar a estrutura do arquivo '.csv' a ser importado.

import csv
import chardet

def ler_csv(nome_arquivo):
       
    
    with open(nome_arquivo, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(100000))
        encoding_arquivo = result['encoding']
    
    
    with open(nome_arquivo, newline='',encoding = encoding_arquivo) as f:
        reader = csv.reader(f)
        try:
            print(reader.line_num)
            for row in reader:
                print(reader.line_num,row)
        except csv.Error as e:
            sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))
    return encoding_arquivo

In [3]:
## Função utilizada como preparação para transformar 
## string (números no formato brasileiro: com pontos nos milhares e vírgula nas decimais) em float.

## Para transformar números no formato texto para float precisamos retirar os pontos;
## então substituir parêntesis por '-', para números negativos; e, por fim
## substituir vígula por ponto para a separação das casas decimais.

def remover_pontos_finais(nome):
    nome = nome.replace('.',"").replace('(',"-").replace(')',"").replace(',',".")
    return nome

In [4]:
## Função utilizada para remover caracteres especiais.

def remover_caracteres_especiais(nome):
    nome = nome.replace('Á','A').replace('Â','A').replace('Ã','A').replace('À','A')\
    .replace('É','E').replace('Ê','E').replace('Í','I').replace('Ó','O').replace('Ô','O')\
    .replace('Õ','O').replace('Ú','U').replace('Ü','U').replace('Ç', 'C')
    return nome

In [5]:
## Função para identificar se há valores fantantes, ou NaN em um dataframe

def identificar_faltantes(df):
    
    #Mostra o tamanho do meu dataset
    print("Shape do dataframe:",df.shape)
    
    #mostra em que coluna estão esses elementos faltantes
    colunas_faltantes = df.columns[df.isna().any()].tolist()
    print("Colunas que contêm valores faltantes:",colunas_faltantes)
    
    #traz o número de linhas que restariam se eu tirasse os que seriam excluídas pelo dropna
    print("Número de linhas com valores faltantes:",df.shape[0]-df.dropna().shape[0],"\n")

    
    return colunas_faltantes

In [6]:
from zipfile import ZipFile

def descompactar(nome_arquivo_compactado):
    # lê o arquivo compactado e extrai o conteúdo
    with ZipFile (nome_arquivo_compactado, 'r') as zip:
        zip.extractall()
        zip.printdir()        
    return "Concluído"

# 2. Coleta das bases de dados

## 2.1. Arrecadação das Receitas Administradas pela RFB, por Município (arrecadacao-da-receita-administrada-pela-rfb-por-municipio-2018.xlsx)

Para esta base, nenhum detalhe adicional para obtenção dos dados é necessário, vez que o download do arquivo é feito diretamente a partir do link https://www.gov.br/receitafederal/pt-br/acesso-a-informacao/dados-abertos/receitadata/arrecadacao/copy_of_arrecadacao-das-receitas-administradas-pela-rfb-por-municipio/arrecadacao-das-receitas-administradas-pela-rfb/arrecadacao-da-receita-administrada-pela-rfb-por-municipio-2018.xlsx.

**O documento possui três planilhas, a de interesse é a denominada 'TOTAL' (que agrega os valores pagos tanto por meio de DARF como por GPS – que são tipos de guias para pagamento).**


- Data da coleta: 19/3/2021;


- Possui metadados: não.



### - Leitura do arquivo '.xls'.

Iniciamos com a visualização do arquivo '.xls' para verificarmos sua estrutura original e analisarmos as linhas que efetivamente deverão ser lidas.

O arquivo possui três planilhas, sendo a que é de nosso interesse é a de nome "TOTAL" (que agrega pagamentos feitos tanto por GPS quanto por DARF).

In [7]:
nome_arquivo = 'arrecadacao-da-receita-administrada-pela-rfb-por-municipio-2018.xlsx'
planilha_interesse = 'TOTAL'
arrecadacao_2018 = pd.read_excel(nome_arquivo,sheet_name=planilha_interesse)
arrecadacao_2018

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,ARRECADAÇÃO DAS RECEITAS ADMINISTRADAS PELA RF...,NaN,NaN
1,TOTAL,NaN,NaN
2,PERÍODO: 2018,NaN,NaN
3,"UNIDADE: R$ 1,00",NaN,NaN
4,MUNICÍPIOS,UF,ARRECADAÇÃO
...,...,...,...
5578,ZE DOCA,MA,1.326e+07
5579,ZERADO,-,1.0381e+10
5580,ZORTEA,SC,4.23176e+06
5581,{Ñ CLASS},-,5.15863e+07


Como se observa, acima, as primeiras linhas do arquivo são apenas informativas e não deverão ser carregadas (assim temos que configurar o parâmetro skiprows quando importarmos o csv).

Vemos também que, no fim do arquivo, há linhas que não nos interessam (totailizações, comentários, notas, etc). Assim, será necessário limitar a importação dos dados, configurando o parâmetro skipfooter (no caso).

Como utilizaremos o skipfooter, é necessário configurar o parâmetro engine='python'

Por fim, há apenas três colunas de interesse (as primeiras) de modo que vamos definir o parâmetro usecols.

In [8]:
arrecadacao_2018 = pd.read_excel(nome_arquivo,sheet_name=planilha_interesse,skiprows=5, skipfooter=2)
arrecadacao_2018

,MUNICÍPIOS,UF,ARRECADAÇÃO
0,ABADIA DE GOIAS,GO,1.483209e+07
1,ABADIA DOS DOURADOS,MG,8.950170e+06
2,ABADIANIA,GO,1.779163e+07
3,ABAETE,MG,3.525459e+07
4,ABAETETUBA,PA,5.956002e+07
...,...,...,...
5571,ZABELE,PB,1.588994e+06
5572,ZACARIAS,SP,3.651978e+06
5573,ZE DOCA,MA,1.325997e+07
5574,ZERADO,-,1.038097e+10


In [9]:
arrecadacao_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MUNICÍPIOS   5576 non-null   object 
 1   UF           5576 non-null   object 
 2   ARRECADAÇÃO  5576 non-null   float64
dtypes: float64(1), object(2)
memory usage: 130.8+ KB


### - Promovendo transformações nos dados

- Criar uma nova coluna, padronizada, chamada 'municipio-uf' que será a chave para joins/merges com outros dataframes;
- Renomear a coluna 'ARRECADAÇÃO' para '01_arrecadacao_2018' e transformá-la para 'float'
- Reordenar as colunas.

In [10]:
## A coluna chamada municipio-uf que será o resultado da 
## concatenação do conteúdo, em maíusculas ".str.upper()", das colunas MUNICIPIO e UF

arrecadacao_2018['municipio-uf']=arrecadacao_2018['MUNICÍPIOS'].str.upper()+'-'+arrecadacao_2018['UF'].str.upper()
arrecadacao_2018.head()

,MUNICÍPIOS,UF,ARRECADAÇÃO,municipio-uf
0,ABADIA DE GOIAS,GO,1.483209e+07,ABADIA DE GOIAS-GO
1,ABADIA DOS DOURADOS,MG,8.950170e+06,ABADIA DOS DOURADOS-MG
2,ABADIANIA,GO,1.779163e+07,ABADIANIA-GO
3,ABAETE,MG,3.525459e+07,ABAETE-MG
4,ABAETETUBA,PA,5.956002e+07,ABAETETUBA-PA


In [11]:
print('Shape',arrecadacao_2018.shape)
print('Valores únicos',pd.Series(arrecadacao_2018['municipio-uf']).unique().shape)
# Veremos que não há duplicidades.

Shape (5576, 4)
Valores únicos (5576,)


In [12]:
# Renomeando a coluna ARRECADAÇAO para "01_arrecadacao_2018".
arrecadacao_2018=arrecadacao_2018.rename({"ARRECADAÇÃO": "01_arrecadacao_2018"}, axis=1,)

In [13]:
## Vamos, por fim, manter apenas as variáveis de interesse, reorndenando-as.
colunas_a_manter = ['municipio-uf','01_arrecadacao_2018']
arrecadacao_2018=arrecadacao_2018[colunas_a_manter]
arrecadacao_2018


,municipio-uf,01_arrecadacao_2018
0,ABADIA DE GOIAS-GO,1.483209e+07
1,ABADIA DOS DOURADOS-MG,8.950170e+06
2,ABADIANIA-GO,1.779163e+07
3,ABAETE-MG,3.525459e+07
4,ABAETETUBA-PA,5.956002e+07
...,...,...
5571,ZABELE-PB,1.588994e+06
5572,ZACARIAS-SP,3.651978e+06
5573,ZE DOCA-MA,1.325997e+07
5574,ZERADO--,1.038097e+10


In [14]:
arrecadacao_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   municipio-uf         5576 non-null   object 
 1   01_arrecadacao_2018  5576 non-null   float64
dtypes: float64(1), object(1)
memory usage: 87.2+ KB


### - Cria '.csv' com o resultado das transformações na base original.

In [15]:
arrecadacao_2018.to_csv('ntbk 01_01 - rfb arrecadacao 2018.csv',index=False)

## 2.2. Tabela 1685 - Unidades locais, empresas e outras organizações atuantes, pessoal ocupado total, pessoal ocupado assalariado, pessoal assalariado médio, salários e outras remunerações e salário médio mensal (tabela1685.csv)

Descrição: 

O Cadastro Central de Empresas - Cempre constitui um importante acervo de dados sobre o universo das empresas e outras organizações formais e suas respectivas unidades locais existentes no Brasil, reunindo informações cadastrais e econômicas oriundas de pesquisas anuais do IBGE, nas áreas de Indústria, Construção, Comércio e Serviços, e de registros administrativos do Ministério do Trabalho e Previdência Social, como a Relação Anual de Informações Sociais - RAIS. 

Tabela 1685 - Unidades locais, empresas e outras organizações atuantes, pessoal ocupado total, pessoal ocupado assalariado, pessoal assalariado médio, salários e outras remunerações e salário médio mensal.


- Fonte: Site do Instituto Brasileiro de Geografia e Estatística na Internet (Sistema IBGE de Recuperação Automática - SIDRA);


- Endereço eletrônico: https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela1685.csv&terr=N&rank=-&query=t/1685/n6/all/v/all/p/last%201/d/v662%200,v1606%201,v5944%202,v10143%202/l/,v,t%2Bp


- Data da coleta: 19/3/2021;


- Possui metadados: sim, mas sem descrição das variáveis (https://metadados.ibge.gov.br/consulta/estatisticos/operacoes-estatisticas/CL).



### - Leitura do arquivo '.csv'.

Iniciamos com a visualização do arquivo '.csv' para verificarmos sua estrutura original e analisarmos as linhas que efetivamente deverão ser lidas.

In [16]:
nome_arquivo = 'tabela1685.csv'
enconding_arquivo = ler_csv(nome_arquivo) #Chama a função para a leitura do arquivo '.csv'

0
1 ['Tabela 1685 - Unidades locais, empresas e outras organizações atuantes, pessoal ocupado total, pessoal ocupado assalariado, pessoal assalariado médio, salários e outras remunerações e salário médio mensal']
2 ['Município', 'Ano', 'Variável']
3 ['Município', 'Ano', 'Número de unidades locais (Unidades)', 'Número de empresas e outras organizações atuantes (Unidades)', 'Pessoal ocupado total (Pessoas)', 'Pessoal ocupado assalariado (Pessoas)', 'Pessoal assalariado médio (Pessoas)', 'Salários e outras remunerações (Mil Reais)', 'Salário médio mensal (Salários mínimos)', 'Salário médio mensal em reais (Reais)']
4 ["Alta Floresta D'Oeste (RO)", '2018', '521', '513', '3271', '2584', '2557.86', '57902', '1.8', '1741.31']
5 ['Ariquemes (RO)', '2018', '2507', '2407', '19886', '16578', '16683.88', '422650', '2.0', '1948.68']
6 ['Cabixi (RO)', '2018', '79', '77', '607', '535', '540.03', '11744', '1.8', '1672.78']
7 ['Cacoal (RO)', '2018', '2230', '2139', '19942', '16815', '17093.05', '431101

517 ['Cururupu (MA)', '2018', '188', '188', '1512', '1322', '1325.74', '33273', '2.0', '1930.57']
518 ['Davinópolis (MA)', '2018', '61', '60', '2078', '2024', '2017.86', '43918', '1.8', '1674.20']
519 ['Dom Pedro (MA)', '2018', '345', '341', '1629', '1290', '1333.40', '27806', '1.7', '1604.10']
520 ['Duque Bacelar (MA)', '2018', '27', '27', '507', '488', '696.96', '16159', '1.9', '1783.45']
521 ['Esperantinópolis (MA)', '2018', '177', '173', '1385', '1204', '1379.51', '25912', '1.5', '1444.87']
522 ['Estreito (MA)', '2018', '490', '473', '3823', '3269', '3479.58', '88045', '2.0', '1946.42']
523 ['Feira Nova do Maranhão (MA)', '2018', '29', '29', '462', '436', '431.31', '9390', '1.8', '1674.68']
524 ['Fernando Falcão (MA)', '2018', '17', '16', '334', '318', '341.26', '6112', '1.4', '1377.65']
525 ['Formosa da Serra Negra (MA)', '2018', '115', '114', '803', '688', '689.95', '18693', '2.2', '2084.08']
526 ['Fortaleza dos Nogueiras (MA)', '2018', '123', '123', '833', '699', '685.87', '1502

945 ['Crato (CE)', '2018', '2137', '2056', '20136', '17926', '17697.51', '416087', '1.9', '1808.54']
946 ['Croatá (CE)', '2018', '163', '158', '944', '814', '828.45', '21204', '2.1', '1968.84']
947 ['Cruz (CE)', '2018', '481', '468', '2110', '1680', '1867.75', '39543', '1.7', '1628.57']
948 ['Deputado Irapuan Pinheiro (CE)', '2018', '103', '101', '1048', '553', '584.95', '12099', '1.7', '1591.06']
949 ['Ererê (CE)', '2018', '29', '29', '503', '478', '466.81', '8741', '1.5', '1440.36']
950 ['Eusébio (CE)', '2018', '2679', '2637', '41970', '37600', '38185.16', '995149', '2.1', '2004.70']
951 ['Farias Brito (CE)', '2018', '183', '180', '1353', '1202', '1198.68', '25702', '1.7', '1649.36']
952 ['Forquilha (CE)', '2018', '211', '207', '2181', '1976', '2212.13', '39907', '1.5', '1387.69']
953 ['Fortaleza (CE)', '2018', '63057', '56922', '849045', '766815', '756510.39', '26613750', '2.8', '2706.12']
954 ['Fortim (CE)', '2018', '182', '180', '1141', '953', '1269.16', '25720', '1.6', '1558.85']

1194 ['Rafael Godeiro (RN)', '2018', '36', '36', '318', '298', '295.93', '6514', '1.8', '1693.28']
1195 ['Riacho da Cruz (RN)', '2018', '28', '28', '270', '252', '242.41', '5566', '1.9', '1766.33']
1196 ['Riacho de Santana (RN)', '2018', '17', '17', '278', '262', '255.61', '6031', '1.9', '1814.97']
1197 ['Riachuelo (RN)', '2018', '49', '49', '419', '370', '378.86', '8755', '1.9', '1777.58']
1198 ['Rodolfo Fernandes (RN)', '2018', '32', '31', '374', '344', '335.73', '6842', '1.6', '1567.68']
1199 ['Tibau (RN)', '2018', '120', '118', '757', '645', '704.66', '14096', '1.6', '1538.72']
1200 ['Ruy Barbosa (RN)', '2018', '11', '11', '303', '296', '289.78', '8541', '2.4', '2267.33']
1201 ['Santa Cruz (RN)', '2018', '560', '531', '3938', '3334', '3323.56', '72160', '1.8', '1670.12']
1202 ['Santana do Matos (RN)', '2018', '130', '124', '1045', '945', '1030.93', '21837', '1.7', '1629.41']
1203 ['Santana do Seridó (RN)', '2018', '46', '46', '486', '436', '442.17', '8745', '1.6', '1521.31']
1204 [

1444 ['Sapé (PB)', '2018', '507', '491', '4500', '4003', '3873.74', '77056', '1.6', '1530.15']
1445 ['São Vicente do Seridó (PB)', '2018', '41', '41', '483', '453', '497.68', '11467', '1.9', '1772.31']
1446 ['Serra Branca (PB)', '2018', '140', '139', '1112', '982', '910.64', '16554', '1.5', '1398.34']
1447 ['Serra da Raiz (PB)', '2018', '31', '31', '339', '316', '334.74', '6370', '1.5', '1463.79']
1448 ['Serra Grande (PB)', '2018', '34', '34', '316', '289', '254.03', '5146', '1.6', '1558.33']
1449 ['Serra Redonda (PB)', '2018', '61', '61', '739', '678', '631.02', '11281', '1.4', '1375.23']
1450 ['Serraria (PB)', '2018', '45', '44', '378', '344', '356.75', '7690', '1.7', '1658.21']
1451 ['Sertãozinho (PB)', '2018', '35', '33', '376', '350', '413.86', '7611', '1.5', '1414.67']
1452 ['Sobrado (PB)', '2018', '60', '60', '605', '554', '541.98', '10670', '1.6', '1514.39']
1453 ['Solânea (PB)', '2018', '487', '477', '2124', '1681', '1747.65', '38865', '1.8', '1710.67']
1454 ['Soledade (PB)', 

...

2943 ['Santa Rosa da Serra (MG)', '2018', '43', '43', '377', '335', '372.83', '8183', '1.8', '1688.32']
2944 ['Santa Vitória (MG)', '2018', '623', '613', '5008', '4325', '4748.18', '145649', '2.5', '2359.59']
2945 ['Santo Antônio do Amparo (MG)', '2018', '339', '332', '2418', '1993', '2215.45', '48341', '1.8', '1678.44']
2946 ['Santo Antônio do Aventureiro (MG)', '2018', '74', '73', '496', '421', '416.37', '8765', '1.7', '1619.29']
2947 ['Santo Antônio do Grama (MG)', '2018', '72', '71', '424', '354', '354.03', '7866', '1.8', '1709.15']
2948 ['Santo Antônio do Itambé (MG)', '2018', '21', '20', '312', '288', '304.51', '6234', '1.7', '1574.90']
2949 ['Santo Antônio do Jacinto (MG)', '2018', '124', '122', '781', '614', '641.20', '12239', '1.5', '1468.23']
2950 ['Santo Antônio do Monte (MG)', '2018', '859', '822', '5862', '4759', '4951.12', '103857', '1.7', '1613.57']
2951 ['Santo Antônio do Retiro (MG)', '2018', '60', '60', '388', '341', '426.25', '7852', '1.5', '1417.07']
2952 ['Santo An

3943 ['Balsa Nova (PR)', '2018', '403', '400', '3588', '3092', '3120.22', '105715', '2.7', '2606.19']
3944 ['Bandeirantes (PR)', '2018', '1189', '1169', '7516', '6143', '6037.09', '159210', '2.1', '2028.61']
3945 ['Barbosa Ferraz (PR)', '2018', '262', '258', '1442', '1137', '1161.11', '28260', '2.0', '1872.18']
3946 ['Barracão (PR)', '2018', '505', '501', '2680', '2019', '2060.62', '49717', '1.9', '1855.94']
3947 ['Barra do Jacaré (PR)', '2018', '43', '42', '313', '274', '269.71', '7048', '2.1', '2010.28']
3948 ['Bela Vista da Caroba (PR)', '2018', '109', '107', '439', '336', '327.08', '8703', '2.1', '2046.89']
3949 ['Bela Vista do Paraíso (PR)', '2018', '466', '451', '2797', '2187', '2192.35', '55247', '2.0', '1938.46']
3950 ['Bituruna (PR)', '2018', '399', '376', '3608', '3210', '3213.04', '78460', '2.0', '1878.39']
3951 ['Boa Esperança (PR)', '2018', '149', '149', '720', '525', '529.08', '16736', '2.6', '2433.30']
3952 ['Boa Esperança do Iguaçu (PR)', '2018', '77', '77', '507', '413

4193 ['Porto Amazonas (PR)', '2018', '122', '114', '1044', '880', '910.15', '24324', '2.2', '2055.80']
4194 ['Porto Barreiro (PR)', '2018', '58', '58', '334', '290', '292.91', '8543', '2.4', '2243.50']
4195 ['Porto Rico (PR)', '2018', '126', '126', '650', '506', '495.24', '14319', '2.3', '2224.13']
4196 ['Porto Vitória (PR)', '2018', '109', '107', '660', '529', '491.02', '13299', '2.2', '2083.49']
4197 ['Prado Ferreira (PR)', '2018', '151', '150', '941', '759', '744.06', '20472', '2.2', '2116.46']
4198 ['Pranchita (PR)', '2018', '266', '263', '1401', '1052', '1059.78', '26188', '2.0', '1900.85']
4199 ['Presidente Castelo Branco (PR)', '2018', '119', '119', '714', '578', '608.66', '15180', '2.0', '1918.49']
4200 ['Primeiro de Maio (PR)', '2018', '233', '229', '1519', '1165', '1161.01', '32315', '2.2', '2141.07']
4201 ['Prudentópolis (PR)', '2018', '1336', '1270', '8159', '6531', '6505.04', '162008', '2.0', '1915.77']
4202 ['Quarto Centenário (PR)', '2018', '93', '90', '686', '588', '572

4442 ['Itapiranga (SC)', '2018', '699', '674', '7578', '6680', '6434.02', '156901', '2.0', '1875.86']
4443 ['Itapoá (SC)', '2018', '812', '791', '5051', '3961', '3625.02', '139053', '3.1', '2950.70']
4444 ['Ituporanga (SC)', '2018', '1093', '1070', '7480', '6189', '6259.47', '169297', '2.2', '2080.51']
4445 ['Jaborá (SC)', '2018', '159', '158', '885', '699', '693.31', '17258', '2.0', '1914.81']
4446 ['Jacinto Machado (SC)', '2018', '312', '298', '2500', '2038', '2133.69', '58497', '2.2', '2108.92']
4447 ['Jaguaruna (SC)', '2018', '676', '668', '4611', '3790', '3766.30', '104949', '2.2', '2143.49']
4448 ['Jaraguá do Sul (SC)', '2018', '8058', '7715', '74641', '64331', '66289.87', '2634473', '3.2', '3057.06']
4449 ['Jardinópolis (SC)', '2018', '85', '85', '293', '219', '225.26', '6475', '2.3', '2211.11']
4450 ['Joaçaba (SC)', '2018', '1910', '1840', '18166', '15873', '16111.97', '528898', '2.6', '2525.11']
4451 ['Joinville (SC)', '2018', '23718', '22314', '242048', '210760', '207490.55',

5574 ['Fonte: IBGE - Cadastro Central de Empresas']
5575 []
5576 ['Notas']
5577 ['1 - Os dados com menos de 3 (três) informantes estão desidentificados com o caracter X.']
5578 ['2 - Esta tabela substitui a tabela 454.']
5579 ['']
5580 ['Legenda']
5581 ['Símbolo', 'Significado']
5583 ['-', 'Zero absoluto, não resultante de um cálculo ou arredondamento.\nEx: Em determinado município não existem pessoas de 14 anos de idade sem instrução.']
5586 ['0', 'Zero resultante de um cálculo ou arredondamento.\nEx: A inflação do feijão em determinada Região Metropolitana foi 0.\nDeterminado município produziu 400 kg de sementes de girassol e os dados da tabela são expressos em toneladas.']
5588 ['X', 'Valor inibido para não identificar o informante.\nEx: Determinado município só possui uma empresa produtora de cimento, logo o valor de sua produção deve ser inibido.']
5590 ['..', 'Valor não se aplica.\nEx: Não se pode obter o total da produção agrícola em determinado município quando os produtos agr

Como se observa, acima, as primeiras 2 linhas do arquivo são apenas informativas e não deverão ser carregadas (assim temos que configurar o parâmetro skiprows = 5 quando importarmos o csv).

Vemos também que, no fim do arquivo, há linhas que não nos interessam (totailizações, comentários, notas, etc). Assim, será necessário limitar a importação dos dados, configurando o parâmetro skipfooter = 14 (no caso).

Como utilizaremos o skipfooter, é necessário configurar o parâmetro engine='python'

In [17]:
IBGE_Empresas=pd.read_csv(nome_arquivo, delimiter=',',engine='python', skiprows=2, skipfooter=14,encoding = enconding_arquivo)
IBGE_Empresas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 10 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Município                                                     5570 non-null   object 
 1   Ano                                                           5570 non-null   int64  
 2   Número de unidades locais (Unidades)                          5570 non-null   int64  
 3   Número de empresas e outras organizações atuantes (Unidades)  5570 non-null   int64  
 4   Pessoal ocupado total (Pessoas)                               5570 non-null   int64  
 5   Pessoal ocupado assalariado (Pessoas)                         5570 non-null   int64  
 6   Pessoal assalariado médio (Pessoas)                           5570 non-null   float64
 7   Salários e outras remunerações (Mil Reais)                    5570 no

In [18]:
IBGE_Empresas

,Município,Ano,Número de unidades locais (Unidades),Número de empresas e outras organizações atuantes (Unidades),Pessoal ocupado total (Pessoas),Pessoal ocupado assalariado (Pessoas),Pessoal assalariado médio (Pessoas),Salários e outras remunerações (Mil Reais),Salário médio mensal (Salários mínimos),Salário médio mensal em reais (Reais)
0,Alta Floresta D'Oeste (RO),2018,521,513,3271,2584,2557.86,57902,1.8,1741.31
1,Ariquemes (RO),2018,2507,2407,19886,16578,16683.88,422650,2.0,1948.68
2,Cabixi (RO),2018,79,77,607,535,540.03,11744,1.8,1672.78
3,Cacoal (RO),2018,2230,2139,19942,16815,17093.05,431101,2.0,1940.07
4,Cerejeiras (RO),2018,389,377,2482,2044,2076.75,52751,2.0,1953.89
...,...,...,...,...,...,...,...,...,...,...
5565,Vianópolis (GO),2018,390,388,2317,1873,1920.39,49687,2.1,1990.26
5566,Vicentinópolis (GO),2018,200,199,1923,1673,1808.01,61111,2.7,2600.00
5567,Vila Boa (GO),2018,66,65,885,825,961.96,22704,1.9,1815.55
5568,Vila Propício (GO),2018,79,76,857,763,756.26,15393,1.6,1565.71


### - Promovendo transformações nos dados

- Selecionar apenas os dados de 2018
- Criar uma nova coluna, padronizada, chamada 'municipio-uf' que será a chave para joins/merges com outros dataframes;
- Atribuir nomes mais sucintos às variáveis;
- Criação de colunas adicionais;
- Reordenar as colunas.

In [19]:
# Vamos agora manter no dataframe exclusivamente o ano de 2018

IBGE_Empresas = IBGE_Empresas.loc[IBGE_Empresas['Ano'] == 2018]
print (IBGE_Empresas.shape)
IBGE_Empresas.head()

(5570, 10)


,Município,Ano,Número de unidades locais (Unidades),Número de empresas e outras organizações atuantes (Unidades),Pessoal ocupado total (Pessoas),Pessoal ocupado assalariado (Pessoas),Pessoal assalariado médio (Pessoas),Salários e outras remunerações (Mil Reais),Salário médio mensal (Salários mínimos),Salário médio mensal em reais (Reais)
0,Alta Floresta D'Oeste (RO),2018,521,513,3271,2584,2557.86,57902,1.8,1741.31
1,Ariquemes (RO),2018,2507,2407,19886,16578,16683.88,422650,2.0,1948.68
2,Cabixi (RO),2018,79,77,607,535,540.03,11744,1.8,1672.78
3,Cacoal (RO),2018,2230,2139,19942,16815,17093.05,431101,2.0,1940.07
4,Cerejeiras (RO),2018,389,377,2482,2044,2076.75,52751,2.0,1953.89


In [20]:
# Criando a nova coluna 'municipio-uf'.
IBGE_Empresas['municipio-uf']=IBGE_Empresas['Município'].str[:-5].str.upper() \
+'-'+IBGE_Empresas['Município'].str[-3:-1].str.upper()

# Removendo caracteres especiais da coluna recém-criada.
IBGE_Empresas['municipio-uf'] = IBGE_Empresas['municipio-uf'].apply(remover_caracteres_especiais)

IBGE_Empresas.head()

,Município,Ano,Número de unidades locais (Unidades),Número de empresas e outras organizações atuantes (Unidades),Pessoal ocupado total (Pessoas),Pessoal ocupado assalariado (Pessoas),Pessoal assalariado médio (Pessoas),Salários e outras remunerações (Mil Reais),Salário médio mensal (Salários mínimos),Salário médio mensal em reais (Reais),municipio-uf
0,Alta Floresta D'Oeste (RO),2018,521,513,3271,2584,2557.86,57902,1.8,1741.31,ALTA FLORESTA D'OESTE-RO
1,Ariquemes (RO),2018,2507,2407,19886,16578,16683.88,422650,2.0,1948.68,ARIQUEMES-RO
2,Cabixi (RO),2018,79,77,607,535,540.03,11744,1.8,1672.78,CABIXI-RO
3,Cacoal (RO),2018,2230,2139,19942,16815,17093.05,431101,2.0,1940.07,CACOAL-RO
4,Cerejeiras (RO),2018,389,377,2482,2044,2076.75,52751,2.0,1953.89,CEREJEIRAS-RO


In [21]:
print('Shape',IBGE_Empresas.shape)
print('Valores únicos',pd.Series(IBGE_Empresas['municipio-uf']).unique().shape)
# Veremos que não há duplicidades.

Shape (5570, 11)
Valores únicos (5570,)


In [22]:
# Vamos também renomear as colunas para nomes mais sucintos.

IBGE_Empresas.columns = ['02_municipio','02_ano','02_unidades','02_unidades_atuantes','02_ocupados_total','02_ocupados_assalariados','02_ocupados_assalariados_medio','02_total_salarios_remuneracoes','02_salario_medio_mensal_em_SM','02_salario_medio_mensal_em_Reais','municipio-uf']

In [23]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Empresas)

Shape do dataframe: (5570, 11)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [24]:
## Vamos criar duas colunas com valores percentuais de algumas colunas 
## (para que tragam uma informação em mesma base independentemente do tamanho do valor nominal)
## Em seguida apagar a coluna que deu origem para não influenciar indevidamente o modelo.

IBGE_Empresas['02_unidades_atuantes_%']=IBGE_Empresas['02_unidades_atuantes']/IBGE_Empresas['02_unidades']*100
IBGE_Empresas = IBGE_Empresas.drop(['02_unidades_atuantes'],1)
 
## Eu iria usar a coluna 02_ocupados_assalariados_medio, em vez de 02_ocupados_assalariados, mas há inconsistências
## naquela coluna que geram valores superiores a 100% quando comparadas com 02_ocupados_total.
## Então vamos usar 02_ocupados_assalariados para criar o %.
#IBGE_Empresas['02_ocupados_assalariados_medio_%']=IBGE_Empresas['02_ocupados_assalariados_medio']/IBGE_Empresas['02_ocupados_total']                                                                                            
#IBGE_Empresas = IBGE_Empresas.drop(['02_ocupados_assalariados_medio'],1)

IBGE_Empresas['02_ocupados_assalariados_%']=IBGE_Empresas['02_ocupados_assalariados']/IBGE_Empresas['02_ocupados_total']*100                                                                                            
IBGE_Empresas = IBGE_Empresas.drop(['02_ocupados_assalariados'],1)


## Eu iria apagar, também, a coluna 02_ocupados_assalariados_médio porque utilizaremos 02_ocupados_assalariados
## Mas, como 02_ocupados_assalariados_médio é importante para o cálculo da coluna 02_total_salarios_remuneracoes
## ela será mantida
## IBGE_Empresas = IBGE_Empresas.drop(['02_ocupados_assalariados_medio'],1)

## Apagar a coluna 02_total_salarios_remuneracoes porque é resultado da operação com outras duas colunas:
## 02_total_salarios_remuneracoes = (02_salario_medio_mensal_em_Reais * 13) * 02_ocupados_assalariados_medio
IBGE_Empresas = IBGE_Empresas.drop(['02_total_salarios_remuneracoes'],1)

##Apagar a coluna '02_municipio', redundante com municipio-uf
IBGE_Empresas = IBGE_Empresas.drop(['02_municipio'],1)

##Apagar a coluna '02_ano', valor fixo em todo o dataset (2018)
IBGE_Empresas = IBGE_Empresas.drop(['02_ano'],1)




In [25]:
## Por fim, reodenar as colunas.
cols= ['municipio-uf', '02_unidades','02_unidades_atuantes_%',
       '02_ocupados_total', '02_ocupados_assalariados_%', '02_ocupados_assalariados_medio',
       '02_salario_medio_mensal_em_SM',
       '02_salario_medio_mensal_em_Reais']

IBGE_Empresas = IBGE_Empresas[cols]

In [26]:
IBGE_Empresas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   municipio-uf                      5570 non-null   object 
 1   02_unidades                       5570 non-null   int64  
 2   02_unidades_atuantes_%            5570 non-null   float64
 3   02_ocupados_total                 5570 non-null   int64  
 4   02_ocupados_assalariados_%        5570 non-null   float64
 5   02_ocupados_assalariados_medio    5570 non-null   float64
 6   02_salario_medio_mensal_em_SM     5570 non-null   float64
 7   02_salario_medio_mensal_em_Reais  5570 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 391.6+ KB


In [27]:
IBGE_Empresas.head()

,municipio-uf,02_unidades,02_unidades_atuantes_%,02_ocupados_total,02_ocupados_assalariados_%,02_ocupados_assalariados_medio,02_salario_medio_mensal_em_SM,02_salario_medio_mensal_em_Reais
0,ALTA FLORESTA D'OESTE-RO,521,98.464491,3271,78.997249,2557.86,1.8,1741.31
1,ARIQUEMES-RO,2507,96.011169,19886,83.365182,16683.88,2.0,1948.68
2,CABIXI-RO,79,97.468354,607,88.138386,540.03,1.8,1672.78
3,CACOAL-RO,2230,95.919283,19942,84.319527,17093.05,2.0,1940.07
4,CEREJEIRAS-RO,389,96.915167,2482,82.352941,2076.75,2.0,1953.89


### - Cria '.csv' com o resultado das transformações na base original.

In [28]:
IBGE_Empresas.to_csv('ntbk 01_02 - ibge empresas 2018.csv',index=False)

## 2.3. Produto Interno Bruto dos Municípios (‘base_de_dados_2010_2018_xls.zip’)

Descrição: 

Fornece estimativas do Produto Interno Bruto - PIB dos Municípios, a preços correntes, e do valor adicionado bruto da Agropecuária, da Indústria, dos Serviços e da Administração, saúde e educação públicas e seguridade social, a preços correntes, compatível com as metodologias das Contas Regionais e Nacionais do Brasil, sendo as estimativas obtidas comparáveis entre si. 


Trata-se de uma arquivo compactado que contém nossa base de interesse: **RELATORIO_DTB_BRASIL_MUNICIPIO.xls**


- Fonte: Site do Instituto Brasileiro de Geografia e Estatística na Internet;  


- Endereço eletrônico: https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/2036-np-produto-interno-bruto-dos-municipios/9088-produto-interno-bruto-dos-municipios.html?=&t=downloads;


- Data da coleta: 19/3/2021;


- Possui metadados: sim, mas sem descrição das variáveis (https://metadados.ibge.gov.br/consulta/estatisticos/operacoes-estatisticas/IO ).


### - Descompactação do arquivo original.

In [29]:
arquivo_a_descompactar = 'base_de_dados_2010_2018_xls.zip'
descompactar(arquivo_a_descompactar) ## Chama a função declarada no início do notebook

File Name                                             Modified             Size
PIB dos Municípios - base de dados 2010-2018.xls 2020-12-11 14:42:38     29066752


'Concluído'

### - Leitura do arquivo '.xls'.

Iniciamos com a visualização do arquivo '.xls' para verificarmos sua estrutura original e analisarmos as linhas que efetivamente deverão ser lidas.

In [30]:
nome_arquivo = 'PIB dos Municípios - base de dados 2010-2018.xls'
IBGE_PIB=pd.read_excel(nome_arquivo)
IBGE_PIB

,Ano,Código da Grande Região,Nome da Grande Região,Código da Unidade da Federação,Sigla da Unidade da Federação,Nome da Unidade da Federação,Código do Município,Nome do Município,Região Metropolitana,Código da Mesorregião,Nome da Mesorregião,Código da Microrregião,Nome da Microrregião,Código da Região Geográfica Imediata,Nome da Região Geográfica Imediata,Município da Região Geográfica Imediata,Código da Região Geográfica Intermediária,Nome da Região Geográfica Intermediária,Município da Região Geográfica Intermediária,Código Concentração Urbana,Nome Concentração Urbana,Tipo Concentração Urbana,Código Arranjo Populacional,Nome Arranjo Populacional,Hierarquia Urbana,Hierarquia Urbana (principais categorias),Código da Região Rural,Nome da Região Rural,Região rural (segundo classificação do núcleo),Amazônia Legal,Semiárido,Cidade-Região de São Paulo,"Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)","Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)","Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)","Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, \na preços correntes\n(R$ 1.000)","Valor adicionado bruto total, \na preços correntes\n(R$ 1.000)","Impostos, líquidos de subsídios, sobre produtos, \na preços correntes\n(R$ 1.000)","Produto Interno Bruto, \na preços correntes\n(R$ 1.000)","Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto
0,2010,1,Norte,11,RO,Rondônia,1100015,Alta Floresta D'Oeste,NaN,1102,Leste Rondoniense,11006,Cacoal,110005,Cacoal,do Entorno,1102,Ji-Paraná,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro Local,Centro Local,1101,Região Rural da Capital Regional de Porto Velho,Região Rural de Capital Regional,Sim,Não,Não,69260.391,16118.534,6.249618e+04,9.324466e+04,2.411198e+05,2.095711e+04,2.620769e+05,10731.18,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
1,2010,1,Norte,11,RO,Rondônia,1100023,Ariquemes,NaN,1102,Leste Rondoniense,11003,Ariquemes,110002,Ariquemes,Polo,1101,Porto Velho,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro Subregional B,Centro Subregional,1101,Região Rural da Capital Regional de Porto Velho,Região Rural de Capital Regional,Sim,Não,Não,73711.643,287138.585,4.949463e+05,3.438677e+05,1.199664e+06,1.650296e+05,1.364694e+06,15103.86,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
2,2010,1,Norte,11,RO,Rondônia,1100031,Cabixi,NaN,1102,Leste Rondoniense,11008,Colorado do Oeste,110006,Vilhena,do Entorno,1102,Ji-Paraná,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro Local,Centro Local,1101,Região Rural da Capital Regional de Porto Velho,Região Rural de Capital Regional,Sim,Não,Não,24300.822,3252.506,1.267721e+04,2.517024e+04,6.540077e+04,4.210342e+03,6.961111e+04,11033.62,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
3,2010,1,Norte,11,RO,Rondônia,1100049,Cacoal,NaN,1102,Leste Rondoniense,11006,Cacoal,110005,Cacoal,Polo,1102,Ji-Paraná,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro Subregional B,Centro Subregional,5105,Região Rural do Centro Sub-regional de Vilhena...,Região Rural de Centro Sub-regional,Sim,Não,Não,95259.203,182051.537,4.654473e+05,2.984543e+05,1.041212e+06,1.452817e+05,1.186494e+06,15095.15,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
4,2010,1,Norte,11,RO,Rondônia,1100056,Cerejeiras,NaN,1102,Leste Rondoniense,11008,Colorado do Oeste,110006,Vilhena,do Entorno,1102,Ji-Paraná,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro de Zona B,Centro de Zona,1101,Região Rural da Capital Regional de Porto Velho,Região Rural de Capital Regio

Como se observa, acima, não há linhas a excluir de modo que os parâmetros skiprows e skipfooter não precisam ser configurados.
O dataframe IBGE_PIB já contém os dados de interesse, portanto.

In [31]:
IBGE_PIB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50115 entries, 0 to 50114
Data columns (total 43 columns):
 #   Column                                                                                                                                             Non-Null Count  Dtype  
---  ------                                                                                                                                             --------------  -----  
 0   Ano                                                                                                                                                50115 non-null  int64  
 1   Código da Grande Região                                                                                                                            50115 non-null  int64  
 2   Nome da Grande Região                                                                                                                              50115 non-null  object 
 3   Código

### - Promovendo transformações nos dados

- Selecionar apenas os dados de 2018
- Criar uma nova coluna, padronizada, chamada 'municipio-uf' que será a chave para joins/merges com outros dataframes;
- Excluir colunas que contêm com valores faltantes e de outras sem relevância;
- Atribuir nomes mais sucintos às variáveis;
- Criação de colunas adicionais;
- Reordenar as colunas.

In [32]:
# Vamos agora manter no dataframe exclusivamente o ano de 2018

IBGE_PIB = IBGE_PIB.loc[IBGE_PIB['Ano'] == 2018].copy()
print (IBGE_PIB.shape)

(5570, 43)


In [33]:
# Criando a nova coluna 'municipio-uf'.
IBGE_PIB['municipio-uf']=IBGE_PIB['Nome do Município'].str.upper()+'-'+IBGE_PIB['Sigla da Unidade da Federação'].str.upper()

# Removendo caracteres especiais da coluna recém-criada.
IBGE_PIB['municipio-uf'] = IBGE_PIB['municipio-uf'].apply(remover_caracteres_especiais)

IBGE_PIB.head()

,Ano,Código da Grande Região,Nome da Grande Região,Código da Unidade da Federação,Sigla da Unidade da Federação,Nome da Unidade da Federação,Código do Município,Nome do Município,Região Metropolitana,Código da Mesorregião,Nome da Mesorregião,Código da Microrregião,Nome da Microrregião,Código da Região Geográfica Imediata,Nome da Região Geográfica Imediata,Município da Região Geográfica Imediata,Código da Região Geográfica Intermediária,Nome da Região Geográfica Intermediária,Município da Região Geográfica Intermediária,Código Concentração Urbana,Nome Concentração Urbana,Tipo Concentração Urbana,Código Arranjo Populacional,Nome Arranjo Populacional,Hierarquia Urbana,Hierarquia Urbana (principais categorias),Código da Região Rural,Nome da Região Rural,Região rural (segundo classificação do núcleo),Amazônia Legal,Semiárido,Cidade-Região de São Paulo,"Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)","Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)","Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)","Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, \na preços correntes\n(R$ 1.000)","Valor adicionado bruto total, \na preços correntes\n(R$ 1.000)","Impostos, líquidos de subsídios, sobre produtos, \na preços correntes\n(R$ 1.000)","Produto Interno Bruto, \na preços correntes\n(R$ 1.000)","Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto,municipio-uf
44545,2018,1,Norte,11,RO,Rondônia,1100015,Alta Floresta D'Oeste,NaN,1102,Leste Rondoniense,11006,Cacoal,110005,Cacoal,do Entorno,1102,Ji-Paraná,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro Local,Centro Local,1101,Região Rural da Capital Regional de Porto Velho,Região Rural de Capital Regional,Sim,Não,Não,165892.171,26369.018,123512.405,155271.086,471044.680,28261.302,499305.982,21552.47,"Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária",ALTA FLORESTA D'OESTE-RO
44546,2018,1,Norte,11,RO,Rondônia,1100023,Ariquemes,NaN,1102,Leste Rondoniense,11003,Ariquemes,110002,Ariquemes,Polo,1101,Porto Velho,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro Subregional B,Centro Subregional,1101,Região Rural da Capital Regional de Porto Velho,Região Rural de Capital Regional,Sim,Não,Não,163120.731,339667.233,1034183.951,674541.006,2211512.920,252260.341,2463773.260,23206.36,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...,ARIQUEMES-RO
44547,2018,1,Norte,11,RO,Rondônia,1100031,Cabixi,NaN,1102,Leste Rondoniense,11008,Colorado do Oeste,110006,Vilhena,do Entorno,1102,Ji-Paraná,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro Local,Centro Local,1101,Região Rural da Capital Regional de Porto Velho,Região Rural de Capital Regional,Sim,Não,Não,62337.414,5178.976,25170.517,42163.897,134850.804,5651.465,140502.269,25837.12,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária","Agricultura, inclusive apoio à agricultura e a...",CABIXI-RO
44548,2018,1,Norte,11,RO,Rondônia,1100049,Cacoal,NaN,1102,Leste Rondoniense,11006,Cacoal,110005,Cacoal,Polo,1102,Ji-Paraná,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro Subregional B,Centro Subregional,5105,Região Rural do Centro Sub-regional de Vilhena...,Região Rural de Centro Sub-regional,Sim,Não,Não,212492.509,222558.359,971609.245,548595.488,1955255.600,225198.416,2180454.016,25708.96,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...,CACOAL-RO
44549,2018,1,Norte,11,RO,Rondônia,1100056,Cerejeiras,NaN,1102,Leste Rondoniense,11008,Colorado do Oeste,110006,Vilhena,do Entorno,1102,Ji-Paraná,do Entorno,NaN,NaN,NaN,NaN,NaN,Centro de Zona B,Centro 

In [34]:
print('Shape',IBGE_PIB.shape)
print('Valores únicos',pd.Series(IBGE_PIB['municipio-uf']).unique().shape)
# Veremos que não há duplicidades.

Shape (5570, 44)
Valores únicos (5570,)


In [35]:
# Chama a função para identificar se há valores faltantes ou NaN.
colunas_faltantes = identificar_faltantes(IBGE_PIB)

Shape do dataframe: (5570, 44)
Colunas que contêm valores faltantes: ['Região Metropolitana', 'Código Concentração Urbana', 'Nome Concentração Urbana', 'Tipo Concentração Urbana', 'Código Arranjo Populacional', 'Nome Arranjo Populacional']
Número de linhas com valores faltantes: 5164 



In [36]:
# Além de possuírem valores nulos, as colunas acima não agregam valor ao modelo e serão excluídas.
IBGE_PIB = IBGE_PIB.drop(colunas_faltantes,1)

# Além delas, também serão excluídas as colunas que se refiram a códigos para os quais os nomes estão em colunas específicas.
# Ex. O Código de mesorregiao será excluído e mantido o Nome da Mesorregiao
# Apenas manterei, temporariamente, o código do município para futuro uso.
colunas_a_excluir=['Código da Grande Região','Código da Unidade da Federação',
      'Código da Mesorregião','Código da Microrregião','Código da Região Geográfica Imediata',
      'Município da Região Geográfica Imediata','Código da Região Geográfica Intermediária',
      'Município da Região Geográfica Intermediária',
      'Hierarquia Urbana','Hierarquia Urbana (principais categorias)',
      'Código da Região Rural','Nome da Região Rural','Região rural (segundo classificação do núcleo)',
      'Amazônia Legal','Semiárido','Cidade-Região de São Paulo']
IBGE_PIB = IBGE_PIB.drop(colunas_a_excluir,1)
IBGE_PIB.columns

Index(['Ano', 'Nome da Grande Região', 'Sigla da Unidade da Federação',
       'Nome da Unidade da Federação', 'Código do Município',
       'Nome do Município', 'Nome da Mesorregião', 'Nome da Microrregião',
       'Nome da Região Geográfica Imediata',
       'Nome da Região Geográfica Intermediária',
       'Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)',
       'Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, \na preços correntes\n(R$ 1.000)',
       'Valor adicionado bruto total, \na preços correntes\n(R$ 1.000)',
       'Impostos, líquidos de subsídios, sobre produtos, \na preços correntes\n(R$ 1.000)',
       'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)',
     

In [37]:
# Vamos também renomear as colunas para nomes mais sucintos...

IBGE_PIB.columns=['03_ano','03_grande_regiao','03_uf','03_nome_uf','03_cod_municipio','03_municipio',
      '03_mesorregiao','03_microregiao','03_regiao_imediata','03_regiao_intermediaria',
      '03_vlr_adic_bruto_agropecuaria','03_vlr_adic_bruto_industria','03_vlr_adic_bruto_servicos',
      '03_vlr_adic_bruto_administracao','03_vlr_adic_bruto_total','03_impostos_sobre_produtos',
      '03_pib','03_pib_per_capita','03_atividade_principal_primeira',
      '03_atividade_principal_segunda','03_atividade_principal_terceira', 'municipio-uf']

#... e agora reorndeá-las.
cols = ['municipio-uf','03_ano','03_grande_regiao','03_nome_uf','03_uf','03_cod_municipio','03_municipio',
      '03_mesorregiao','03_microregiao','03_regiao_imediata','03_regiao_intermediaria',
      '03_vlr_adic_bruto_agropecuaria','03_vlr_adic_bruto_industria','03_vlr_adic_bruto_servicos',
      '03_vlr_adic_bruto_administracao','03_vlr_adic_bruto_total','03_impostos_sobre_produtos',
      '03_pib','03_pib_per_capita','03_atividade_principal_primeira',
      '03_atividade_principal_segunda','03_atividade_principal_terceira']

IBGE_PIB = IBGE_PIB[cols]

print(IBGE_PIB.shape)
print(IBGE_PIB.columns)
IBGE_PIB.head()

(5570, 22)
Index(['municipio-uf', '03_ano', '03_grande_regiao', '03_nome_uf', '03_uf',
       '03_cod_municipio', '03_municipio', '03_mesorregiao', '03_microregiao',
       '03_regiao_imediata', '03_regiao_intermediaria',
       '03_vlr_adic_bruto_agropecuaria', '03_vlr_adic_bruto_industria',
       '03_vlr_adic_bruto_servicos', '03_vlr_adic_bruto_administracao',
       '03_vlr_adic_bruto_total', '03_impostos_sobre_produtos', '03_pib',
       '03_pib_per_capita', '03_atividade_principal_primeira',
       '03_atividade_principal_segunda', '03_atividade_principal_terceira'],
      dtype='object')


,municipio-uf,03_ano,03_grande_regiao,03_nome_uf,03_uf,03_cod_municipio,03_municipio,03_mesorregiao,03_microregiao,03_regiao_imediata,03_regiao_intermediaria,03_vlr_adic_bruto_agropecuaria,03_vlr_adic_bruto_industria,03_vlr_adic_bruto_servicos,03_vlr_adic_bruto_administracao,03_vlr_adic_bruto_total,03_impostos_sobre_produtos,03_pib,03_pib_per_capita,03_atividade_principal_primeira,03_atividade_principal_segunda,03_atividade_principal_terceira
44545,ALTA FLORESTA D'OESTE-RO,2018,Norte,Rondônia,RO,1100015,Alta Floresta D'Oeste,Leste Rondoniense,Cacoal,Cacoal,Ji-Paraná,165892.171,26369.018,123512.405,155271.086,471044.680,28261.302,499305.982,21552.47,"Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária"
44546,ARIQUEMES-RO,2018,Norte,Rondônia,RO,1100023,Ariquemes,Leste Rondoniense,Ariquemes,Ariquemes,Porto Velho,163120.731,339667.233,1034183.951,674541.006,2211512.920,252260.341,2463773.260,23206.36,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...
44547,CABIXI-RO,2018,Norte,Rondônia,RO,1100031,Cabixi,Leste Rondoniense,Colorado do Oeste,Vilhena,Ji-Paraná,62337.414,5178.976,25170.517,42163.897,134850.804,5651.465,140502.269,25837.12,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária","Agricultura, inclusive apoio à agricultura e a..."
44548,CACOAL-RO,2018,Norte,Rondônia,RO,1100049,Cacoal,Leste Rondoniense,Cacoal,Cacoal,Ji-Paraná,212492.509,222558.359,971609.245,548595.488,1955255.600,225198.416,2180454.016,25708.96,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...
44549,CEREJEIRAS-RO,2018,Norte,Rondônia,RO,1100056,Cerejeiras,Leste Rondoniense,Colorado do Oeste,Vilhena,Ji-Paraná,61973.278,22818.149,208728.017,109796.030,403315.475,68863.359,472178.834,28714.35,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...


In [38]:
## Vamos criar variáveis com valores percentuais de outras já existentes
## (para que tragam uma informação em mesma base independentemente do tamanho do valor nominal)
## Em seguida apagar a coluna que deu origem para não influenciar indevidamente o modelo.

IBGE_PIB['03_vlr_adic_bruto_agropecuaria_%']=pd.to_numeric(IBGE_PIB['03_vlr_adic_bruto_agropecuaria'],errors='coerce')/pd.to_numeric(IBGE_PIB['03_vlr_adic_bruto_total'],errors='raise')*100
IBGE_PIB = IBGE_PIB.drop(['03_vlr_adic_bruto_agropecuaria'],1)
                                                                                            
IBGE_PIB['03_vlr_adic_bruto_industria_%']=pd.to_numeric(IBGE_PIB['03_vlr_adic_bruto_industria'],errors='coerce')/pd.to_numeric(IBGE_PIB['03_vlr_adic_bruto_total'],errors='raise')*100
IBGE_PIB = IBGE_PIB.drop(['03_vlr_adic_bruto_industria'],1)

IBGE_PIB['03_vlr_adic_bruto_servicos_%']=pd.to_numeric(IBGE_PIB['03_vlr_adic_bruto_servicos'],errors='coerce')/pd.to_numeric(IBGE_PIB['03_vlr_adic_bruto_total'],errors='raise')*100
IBGE_PIB = IBGE_PIB.drop(['03_vlr_adic_bruto_servicos'],1)

IBGE_PIB['03_vlr_adic_bruto_administracao_%']=pd.to_numeric(IBGE_PIB['03_vlr_adic_bruto_administracao'],errors='coerce')/pd.to_numeric(IBGE_PIB['03_vlr_adic_bruto_total'],errors='raise')*100
IBGE_PIB = IBGE_PIB.drop(['03_vlr_adic_bruto_administracao'],1)

# Transforma algumas colunas em numérico
IBGE_PIB['03_impostos_sobre_produtos'] = pd.to_numeric(IBGE_PIB['03_impostos_sobre_produtos'],errors='raise')
IBGE_PIB['03_pib'] = pd.to_numeric(IBGE_PIB['03_pib'],errors='raise')
IBGE_PIB['03_pib_per_capita'] = pd.to_numeric(IBGE_PIB['03_pib_per_capita'],errors='raise')

# Transforma algumas colunas em string
IBGE_PIB['03_cod_municipio'] = IBGE_PIB['03_cod_municipio'].astype(str)

##Apagar a coluna '02_municipio', redundante com municipio-uf
IBGE_PIB = IBGE_PIB.drop(['03_municipio'],1)

# Apagar a coluna '03_ano' pois tem valor fixo (2018) por todo o dataset.
IBGE_PIB = IBGE_PIB.drop(['03_ano'],1)

# Reordenar as colunas
cols=['municipio-uf', '03_nome_uf', '03_uf', '03_cod_municipio', '03_grande_regiao', 
       '03_mesorregiao', '03_microregiao', '03_regiao_imediata',
       '03_regiao_intermediaria', '03_vlr_adic_bruto_total', '03_vlr_adic_bruto_agropecuaria_%',
       '03_vlr_adic_bruto_industria_%', '03_vlr_adic_bruto_servicos_%',
       '03_vlr_adic_bruto_administracao_%',
       '03_impostos_sobre_produtos', '03_pib', '03_pib_per_capita',
       '03_atividade_principal_primeira', '03_atividade_principal_segunda',
       '03_atividade_principal_terceira']
IBGE_PIB=IBGE_PIB[cols]

print (IBGE_PIB.shape)                                                                                                                
print (IBGE_PIB.columns)                                                                                                                

(5570, 20)
Index(['municipio-uf', '03_nome_uf', '03_uf', '03_cod_municipio',
       '03_grande_regiao', '03_mesorregiao', '03_microregiao',
       '03_regiao_imediata', '03_regiao_intermediaria',
       '03_vlr_adic_bruto_total', '03_vlr_adic_bruto_agropecuaria_%',
       '03_vlr_adic_bruto_industria_%', '03_vlr_adic_bruto_servicos_%',
       '03_vlr_adic_bruto_administracao_%', '03_impostos_sobre_produtos',
       '03_pib', '03_pib_per_capita', '03_atividade_principal_primeira',
       '03_atividade_principal_segunda', '03_atividade_principal_terceira'],
      dtype='object')


In [39]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_PIB)
#vendo as linhas que contêm variáveis com valores nulos.
print("Linhas que contém valores nulos:")
IBGE_PIB[pd.isnull(IBGE_PIB).any(axis=1)]

Shape do dataframe: (5570, 20)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 

Linhas que contém valores nulos:


,municipio-uf,03_nome_uf,03_uf,03_cod_municipio,03_grande_regiao,03_mesorregiao,03_microregiao,03_regiao_imediata,03_regiao_intermediaria,03_vlr_adic_bruto_total,03_vlr_adic_bruto_agropecuaria_%,03_vlr_adic_bruto_industria_%,03_vlr_adic_bruto_servicos_%,03_vlr_adic_bruto_administracao_%,03_impostos_sobre_produtos,03_pib,03_pib_per_capita,03_atividade_principal_primeira,03_atividade_principal_segunda,03_atividade_principal_terceira


In [40]:
# investigando os dados originais, vemos que os valores da coluna 03_vlr_adic_bruto_agropecuaria 
# para os municípios acima eram '-'.
# Assim, ao fazer a divisão que deu origem à coluna 03_vlr_adic_bruto_agropecuaria_%, o resultado gerado foi NaN.
# O que temos a fazer agora é substituir NaN por 0 (zero).

IBGE_PIB['03_vlr_adic_bruto_agropecuaria_%'] = IBGE_PIB['03_vlr_adic_bruto_agropecuaria_%'].fillna(0)

In [41]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_PIB)

Shape do dataframe: (5570, 20)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [42]:
IBGE_PIB.head()

,municipio-uf,03_nome_uf,03_uf,03_cod_municipio,03_grande_regiao,03_mesorregiao,03_microregiao,03_regiao_imediata,03_regiao_intermediaria,03_vlr_adic_bruto_total,03_vlr_adic_bruto_agropecuaria_%,03_vlr_adic_bruto_industria_%,03_vlr_adic_bruto_servicos_%,03_vlr_adic_bruto_administracao_%,03_impostos_sobre_produtos,03_pib,03_pib_per_capita,03_atividade_principal_primeira,03_atividade_principal_segunda,03_atividade_principal_terceira
44545,ALTA FLORESTA D'OESTE-RO,Rondônia,RO,1100015,Norte,Leste Rondoniense,Cacoal,Cacoal,Ji-Paraná,471044.680,35.217927,5.597987,26.220953,32.963133,28261.302,499305.982,21552.47,"Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária"
44546,ARIQUEMES-RO,Rondônia,RO,1100023,Norte,Leste Rondoniense,Ariquemes,Ariquemes,Porto Velho,2211512.920,7.375979,15.359044,46.763640,30.501337,252260.341,2463773.260,23206.36,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...
44547,CABIXI-RO,Rondônia,RO,1100031,Norte,Leste Rondoniense,Colorado do Oeste,Vilhena,Ji-Paraná,134850.804,46.226950,3.840523,18.665456,31.267071,5651.465,140502.269,25837.12,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária","Agricultura, inclusive apoio à agricultura e a..."
44548,CACOAL-RO,Rondônia,RO,1100049,Norte,Leste Rondoniense,Cacoal,Cacoal,Ji-Paraná,1955255.600,10.867761,11.382571,49.692186,28.057482,225198.416,2180454.016,25708.96,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...
44549,CEREJEIRAS-RO,Rondônia,RO,1100056,Norte,Leste Rondoniense,Colorado do Oeste,Vilhena,Ji-Paraná,403315.475,15.365956,5.657643,51.753039,27.223362,68863.359,472178.834,28714.35,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...


### - Cria '.csv' com o resultado das transformações na base original.

In [43]:
IBGE_PIB.to_csv('ntbk 01_03 - ibge pib 2018.csv',index=False)

## 2.4. Tabela 6579 - População residente estimada (tabela6579.csv)


Descrição: 

As estimativas populacionais têm fundamental importância para o cálculo de indicadores sociodemográficos nos períodos intercensitários, bem como alimentam as bases de informações de Ministérios e Secretarias Estaduais e Municipais da área social para a implementação de políticas públicas e posterior avaliação de seus respectivos programas. Além disso, em cumprimento ao dispositivo constitucional, as estimativas da população constituem o principal parâmetro para a distribuição, conduzida pelo Tribunal de Contas da União, das quotas partes relativas ao Fundo de Participação de Estados e Municípios.


- Fonte: Site do Instituto Brasileiro de Geografia e Estatística na Internet (Sistema IBGE de Recuperação Automática - SIDRA);


- Endereço eletrônico: https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela6579.csv&terr=N&rank=-&query=t/6579/n6/all/v/all/p/2018/l/,v,t%2Bp
 

- Data da coleta: 19/3/2021;


- Possui metadados: (https://metadados.ibge.gov.br/consulta/estatisticos/operacoes-estatisticas/XF).


### - Leitura do arquivo '.csv'.

Iniciamos com a visualização do arquivo '.csv' para verificarmos sua estrutura original e analisarmos as linhas que efetivamente deverão ser lidas.

In [44]:
nome_arquivo = 'tabela6579.csv'
enconding_arquivo = ler_csv(nome_arquivo) #Chama a função para a leitura do arquivo '.csv'

0
1 ['Tabela 6579 - População residente estimada']
2 ['Município', 'Ano', 'Variável']
3 ['Município', 'Ano', 'População residente estimada (Pessoas)']
4 ["Alta Floresta D'Oeste (RO)", '2018', '23167']
5 ['Ariquemes (RO)', '2018', '106168']
6 ['Cabixi (RO)', '2018', '5438']
7 ['Cacoal (RO)', '2018', '84813']
8 ['Cerejeiras (RO)', '2018', '16444']
9 ['Colorado do Oeste (RO)', '2018', '16227']
10 ['Corumbiara (RO)', '2018', '7567']
11 ['Costa Marques (RO)', '2018', '17855']
12 ["Espigão D'Oeste (RO)", '2018', '32047']
13 ['Guajará-Mirim (RO)', '2018', '45783']
14 ['Jaru (RO)', '2018', '51933']
15 ['Ji-Paraná (RO)', '2018', '127907']
16 ["Machadinho D'Oeste (RO)", '2018', '39097']
17 ["Nova Brasilândia D'Oeste (RO)", '2018', '20459']
18 ['Ouro Preto do Oeste (RO)', '2018', '36340']
19 ['Pimenta Bueno (RO)', '2018', '36434']
20 ['Porto Velho (RO)', '2018', '519531']
21 ['Presidente Médici (RO)', '2018', '19409']
22 ['Rio Crespo (RO)', '2018', '3723']
23 ['Rolim de Moura (RO)', '2018', '5470

984 ['Itarema (CE)', '2018', '41445']
985 ['Itatira (CE)', '2018', '20786']
986 ['Jaguaretama (CE)', '2018', '18102']
987 ['Jaguaribara (CE)', '2018', '11359']
988 ['Jaguaribe (CE)', '2018', '34729']
989 ['Jaguaruana (CE)', '2018', '33967']
990 ['Jardim (CE)', '2018', '27284']
991 ['Jati (CE)', '2018', '7902']
992 ['Jijoca de Jericoacoara (CE)', '2018', '19587']
993 ['Juazeiro do Norte (CE)', '2018', '271926']
994 ['Jucás (CE)', '2018', '24773']
995 ['Lavras da Mangabeira (CE)', '2018', '31584']
996 ['Limoeiro do Norte (CE)', '2018', '59278']
997 ['Madalena (CE)', '2018', '19906']
998 ['Maracanaú (CE)', '2018', '226128']
999 ['Maranguape (CE)', '2018', '127098']
1000 ['Marco (CE)', '2018', '27127']
1001 ['Martinópole (CE)', '2018', '11143']
1002 ['Massapê (CE)', '2018', '38424']
1003 ['Mauriti (CE)', '2018', '46854']
1004 ['Meruoca (CE)', '2018', '15030']
1005 ['Milagres (CE)', '2018', '28466']
1006 ['Milhã (CE)', '2018', '13241']
1007 ['Miraíma (CE)', '2018', '13669']
1008 ['Missão Ve

1983 ['Ibiquera (BA)', '2018', '4043']
1984 ['Ibirapitanga (BA)', '2018', '23343']
1985 ['Ibirapuã (BA)', '2018', '8581']
1986 ['Ibirataia (BA)', '2018', '15760']
1987 ['Ibitiara (BA)', '2018', '16272']
1988 ['Ibititá (BA)', '2018', '17150']
1989 ['Ibotirama (BA)', '2018', '26846']
1990 ['Ichu (BA)', '2018', '6194']
1991 ['Igaporã (BA)', '2018', '15630']
1992 ['Igrapiúna (BA)', '2018', '13367']
1993 ['Iguaí (BA)', '2018', '26868']
1994 ['Ilhéus (BA)', '2018', '164844']
1995 ['Inhambupe (BA)', '2018', '39499']
1996 ['Ipecaetá (BA)', '2018', '14625']
1997 ['Ipiaú (BA)', '2018', '45823']
1998 ['Ipirá (BA)', '2018', '59763']
1999 ['Ipupiara (BA)', '2018', '9817']
2000 ['Irajuba (BA)', '2018', '7243']
2001 ['Iramaia (BA)', '2018', '8893']
2002 ['Iraquara (BA)', '2018', '24942']
2003 ['Irará (BA)', '2018', '28892']
2004 ['Irecê (BA)', '2018', '72386']
2005 ['Itabela (BA)', '2018', '30413']
2006 ['Itaberaba (BA)', '2018', '64325']
2007 ['Itabuna (BA)', '2018', '212740']
2008 ['Itacaré (BA)', 

2983 ['São João Evangelista (MG)', '2018', '15781']
2984 ['São João Nepomuceno (MG)', '2018', '26272']
2985 ['São Joaquim de Bicas (MG)', '2018', '30989']
2986 ['São José da Barra (MG)', '2018', '7371']
2987 ['São José da Lapa (MG)', '2018', '23385']
2988 ['São José da Safira (MG)', '2018', '4255']
2989 ['São José da Varginha (MG)', '2018', '4927']
2990 ['São José do Alegre (MG)', '2018', '4183']
2991 ['São José do Divino (MG)', '2018', '3865']
2992 ['São José do Goiabal (MG)', '2018', '5454']
2993 ['São José do Jacuri (MG)', '2018', '6477']
2994 ['São José do Mantimento (MG)', '2018', '2775']
2995 ['São Lourenço (MG)', '2018', '45488']
2996 ['São Miguel do Anta (MG)', '2018', '6933']
2997 ['São Pedro da União (MG)', '2018', '4709']
2998 ['São Pedro dos Ferros (MG)', '2018', '7858']
2999 ['São Pedro do Suaçuí (MG)', '2018', '5291']
3000 ['São Romão (MG)', '2018', '12139']
3001 ['São Roque de Minas (MG)', '2018', '7026']
3002 ['São Sebastião da Bela Vista (MG)', '2018', '5455']
3003 ['S

4233 ['Salto do Lontra (PR)', '2018', '14695']
4234 ['Santa Amélia (PR)', '2018', '3385']
4235 ['Santa Cecília do Pavão (PR)', '2018', '3376']
4236 ['Santa Cruz de Monte Castelo (PR)', '2018', '7849']
4237 ['Santa Fé (PR)', '2018', '11885']
4238 ['Santa Helena (PR)', '2018', '26206']
4239 ['Santa Inês (PR)', '2018', '1624']
4240 ['Santa Isabel do Ivaí (PR)', '2018', '8603']
4241 ['Santa Izabel do Oeste (PR)', '2018', '14521']
4242 ['Santa Lúcia (PR)', '2018', '3834']
4243 ['Santa Maria do Oeste (PR)', '2018', '9824']
4244 ['Santa Mariana (PR)', '2018', '11829']
4245 ['Santa Mônica (PR)', '2018', '3944']
4246 ['Santana do Itararé (PR)', '2018', '5031']
4247 ['Santa Tereza do Oeste (PR)', '2018', '10182']
4248 ['Santa Terezinha de Itaipu (PR)', '2018', '23224']
4249 ['Santo Antônio da Platina (PR)', '2018', '45728']
4250 ['Santo Antônio do Caiuá (PR)', '2018', '2656']
4251 ['Santo Antônio do Paraíso (PR)', '2018', '2144']
4252 ['Santo Antônio do Sudoeste (PR)', '2018', '20069']
4253 ['Sa

5483 ['Nerópolis (GO)', '2018', '29293']
5484 ['Niquelândia (GO)', '2018', '46039']
5485 ['Nova América (GO)', '2018', '2346']
5486 ['Nova Aurora (GO)', '2018', '2199']
5487 ['Nova Crixás (GO)', '2018', '12791']
5488 ['Nova Glória (GO)', '2018', '8218']
5489 ['Nova Iguaçu de Goiás (GO)', '2018', '2923']
5490 ['Nova Roma (GO)', '2018', '3293']
5491 ['Nova Veneza (GO)', '2018', '9684']
5492 ['Novo Brasil (GO)', '2018', '2985']
5493 ['Novo Gama (GO)', '2018', '113679']
5494 ['Novo Planalto (GO)', '2018', '4445']
5495 ['Orizona (GO)', '2018', '15502']
5496 ['Ouro Verde de Goiás (GO)', '2018', '3797']
5497 ['Ouvidor (GO)', '2018', '6549']
5498 ['Padre Bernardo (GO)', '2018', '33228']
5499 ['Palestina de Goiás (GO)', '2018', '3460']
5500 ['Palmeiras de Goiás (GO)', '2018', '28313']
5501 ['Palmelo (GO)', '2018', '2380']
5502 ['Palminópolis (GO)', '2018', '3589']
5503 ['Panamá (GO)', '2018', '2627']
5504 ['Paranaiguara (GO)', '2018', '9972']
5505 ['Paraúna (GO)', '2018', '10995']
5506 ['Perolâ

Como se observa, acima, as primeiras 3 linhas do arquivo são apenas informativas e não deverão ser carregadas (assim temos que configurar o parâmetro skiprows = 5 quando importarmos o csv).

Vemos também que, no fim do arquivo, há linhas que não nos interessam (totailizações, comentários, notas, etc). Assim, será necessário limitar a importação dos dados, configurando o parâmetro skipfooter = 97 (no caso).

Como utilizaremos o skipfooter, é necessário configurar o parâmetro engine='python'

In [45]:
IBGE_Residentes=pd.read_csv(nome_arquivo, delimiter = ',',engine='python', skiprows=2, skipfooter=97,encoding = enconding_arquivo)
IBGE_Residentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 3 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Município                               5570 non-null   object
 1   Ano                                     5570 non-null   int64 
 2   População residente estimada (Pessoas)  5570 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 130.7+ KB


In [46]:
IBGE_Residentes

,Município,Ano,População residente estimada (Pessoas)
0,Alta Floresta D'Oeste (RO),2018,23167
1,Ariquemes (RO),2018,106168
2,Cabixi (RO),2018,5438
3,Cacoal (RO),2018,84813
4,Cerejeiras (RO),2018,16444
...,...,...,...
5565,Vianópolis (GO),2018,13746
5566,Vicentinópolis (GO),2018,8611
5567,Vila Boa (GO),2018,6026
5568,Vila Propício (GO),2018,5758


### - Promovendo transformações nos dados

- Selecionar apenas os dados de 2018
- Criar uma nova coluna, padronizada, chamada 'municipio-uf' que será a chave para joins/merges com outros dataframes;
- Atribuir nomes mais sucintos às variáveis;
- Reordenar as colunas.

In [47]:
# Criando a nova coluna 'municipio-uf'.
IBGE_Residentes['municipio-uf']=IBGE_Residentes['Município'].str[:-5].str.upper()+'-'+IBGE_Residentes['Município'].str[-3:-1].str.upper()

# Removendo caracteres especiais da coluna recém-criada.
IBGE_Residentes['municipio-uf'] = IBGE_Residentes['municipio-uf'].apply(remover_caracteres_especiais)

In [48]:
# Vamos excluir a coluna Município (que deu origem à municipio-uf) e reodenar.

IBGE_Residentes=IBGE_Residentes.drop(['Município','Ano'],1)

# Renomear a coluna '2018'
IBGE_Residentes = IBGE_Residentes.rename({'População residente estimada (Pessoas)': '04_populacao_residente_2018'}, axis = 1)

#... e agora reorndeá-las.
cols = ['municipio-uf','04_populacao_residente_2018']
IBGE_Residentes = IBGE_Residentes[cols]

print(IBGE_Residentes.shape)
print(IBGE_Residentes.columns)
IBGE_Residentes.head()

(5570, 2)
Index(['municipio-uf', '04_populacao_residente_2018'], dtype='object')


,municipio-uf,04_populacao_residente_2018
0,ALTA FLORESTA D'OESTE-RO,23167
1,ARIQUEMES-RO,106168
2,CABIXI-RO,5438
3,CACOAL-RO,84813
4,CEREJEIRAS-RO,16444


In [49]:
print('Shape',IBGE_Residentes.shape)
print('Valores únicos',pd.Series(IBGE_Residentes['municipio-uf']).unique().shape)
# Veremos que não há duplicidades.

Shape (5570, 2)
Valores únicos (5570,)


### - Cria '.csv' com o resultado das transformações na base original.

In [50]:
IBGE_Residentes.to_csv('ntbk 01_04 - ibge população residente 2018.csv',index=False)

## 2.5. Tabela 200 - População residente, por sexo, situação e grupos de idade (tabela200.csv)

Descrição:

Censo Demográfico - 2010 O Censo Demográfico tem por objetivo contar os habitantes do território nacional, identificar suas características e revelar como vivem os brasileiros, produzindo informações imprescindíveis para a definição de políticas públicas e a tomada de decisões de investimentos da iniciativa privada ou de qualquer nível de governo. Também constitui a única fonte de referência sobre a situação de vida da população nos municípios e em seus recortes internos, como distritos, bairros e localidades, rurais ou urbanas, cujas realidades dependem de seus resultados para serem conhecidas e terem seus dados atualizados.

Tabela 200 - População residente, por sexo, situação e grupos de idade - Amostra - Características Gerais da População


- Fonte: Site do Instituto Brasileiro de Geografia e Estatística na Internet (Sistema IBGE de Recuperação Automática - SIDRA);


- Endereço eletrônico: 
[https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela200.csv&terr=N&rank=-&query=t/200/n6/all/u/y/v/allxp/p/last%201/c2/0/c1/0/c58/0,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,6802,6803,92963,92964,92965/d/v93%200/l/v,c58,t%2Bp%2Bc1%2Bc2]

 
- Data da coleta: 19/3/2021;


- Possui metadados: sim, mas sem descrição das variáveis (https://metadados.ibge.gov.br/consulta/estatisticos/operacoes-estatisticas/CD).


### - Leitura do arquivo '.csv'.

Iniciamos com a visualização do arquivo '.csv' para verificarmos sua estrutura original e analisarmos as linhas que efetivamente deverão ser lidas.

In [51]:
nome_arquivo = 'tabela200.csv'
enconding_arquivo = ler_csv(nome_arquivo) #Chama a função para a leitura do arquivo '.csv'

0
1 ['Tabela 200 - População residente, por sexo, situação e grupos de idade - Amostra - Características Gerais da População']
2 ['Variável - População residente (Pessoas)']
3 ['Município', 'Ano', 'Situação do domicílio', 'Sexo', 'Grupo de idade']
4 ['Município', 'Ano', 'Situação do domicílio', 'Sexo', 'Total', '0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos', '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos', '40 a 44 anos', '45 a 49 anos', '50 a 54 anos', '55 a 59 anos', '60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '75 a 79 anos', '80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos', '100 anos ou mais']
5 ["Alta Floresta D'Oeste (RO)", '2010', 'Total', 'Total', '24392', '1851', '2107', '2401', '2588', '2200', '2092', '1867', '1795', '1715', '1566', '1178', '888', '676', '551', '457', '287', '115', '35', '10', '12', '-']
6 ['Ariquemes (RO)', '2010', 'Total', 'Total', '90353', '7342', '8286', '9256', '9041', '8981', '8458', '7668', '6944', '6416', '5

395 ['Natividade (TO)', '2010', 'Total', 'Total', '9000', '802', '790', '984', '947', '776', '704', '629', '573', '546', '486', '415', '339', '292', '229', '145', '174', '89', '58', '14', '8', '-']
396 ['Nazaré (TO)', '2010', 'Total', 'Total', '4386', '365', '447', '449', '402', '362', '344', '327', '285', '234', '223', '232', '135', '150', '141', '115', '81', '67', '12', '10', '3', '-']
397 ['Nova Olinda (TO)', '2010', 'Total', 'Total', '10686', '1084', '1077', '1118', '1032', '960', '877', '822', '656', '629', '522', '462', '401', '309', '295', '231', '103', '62', '24', '10', '5', '6']
398 ['Nova Rosalândia (TO)', '2010', 'Total', 'Total', '3770', '293', '361', '408', '321', '290', '240', '295', '230', '232', '235', '158', '206', '135', '107', '106', '81', '39', '13', '21', '-', '-']
399 ['Novo Acordo (TO)', '2010', 'Total', 'Total', '3762', '352', '408', '432', '446', '296', '281', '229', '207', '221', '212', '154', '137', '113', '113', '61', '51', '31', '9', '9', '-', '-']
400 ['No

786 ['Lagoa do Piauí (PI)', '2010', 'Total', 'Total', '3863', '328', '329', '395', '369', '393', '341', '300', '252', '193', '205', '130', '208', '115', '101', '98', '51', '21', '17', '13', '6', '-']
787 ['Lagoa do Sítio (PI)', '2010', 'Total', 'Total', '4850', '407', '419', '486', '437', '356', '372', '380', '323', '297', '295', '258', '216', '197', '113', '122', '48', '80', '31', '10', '-', '2']
788 ['Landri Sales (PI)', '2010', 'Total', 'Total', '5281', '435', '495', '525', '437', '421', '365', '360', '336', '312', '302', '331', '210', '210', '184', '143', '114', '47', '28', '10', '9', '6']
789 ['Luís Correia (PI)', '2010', 'Total', 'Total', '28406', '2512', '2623', '2947', '2926', '2825', '2536', '2314', '1952', '1652', '1371', '1055', '1038', '855', '660', '506', '309', '214', '80', '12', '20', '-']
790 ['Luzilândia (PI)', '2010', 'Total', 'Total', '24721', '2233', '2261', '2413', '2618', '2519', '2095', '1718', '1373', '1327', '1175', '959', '1198', '669', '794', '582', '412', '1

1035 ['Pindoretama (CE)', '2010', 'Total', 'Total', '18683', '1421', '1390', '1808', '1875', '1719', '1604', '1506', '1430', '1281', '960', '838', '589', '580', '519', '467', '288', '229', '126', '26', '15', '12']
1036 ['Piquet Carneiro (CE)', '2010', 'Total', 'Total', '15467', '976', '1138', '1396', '1440', '1368', '1218', '1219', '1009', '1009', '892', '783', '719', '623', '542', '414', '305', '187', '165', '38', '26', '-']
1037 ['Pires Ferreira (CE)', '2010', 'Total', 'Total', '10216', '806', '890', '1162', '1157', '780', '660', '622', '677', '604', '600', '460', '363', '288', '350', '278', '215', '176', '104', '24', '-', '-']
1038 ['Poranga (CE)', '2010', 'Total', 'Total', '12001', '995', '1089', '1434', '1268', '943', '881', '827', '717', '682', '587', '537', '457', '450', '353', '361', '163', '133', '71', '41', '4', '7']
1039 ['Porteiras (CE)', '2010', 'Total', 'Total', '15061', '1282', '1233', '1523', '1607', '1453', '1215', '1117', '928', '777', '763', '650', '578', '550', '433

1285 ['Brejo dos Santos (PB)', '2010', 'Total', 'Total', '6198', '453', '520', '558', '525', '557', '465', '447', '434', '383', '379', '314', '287', '270', '178', '163', '79', '87', '34', '48', '17', '-']
1286 ['Caaporã (PB)', '2010', 'Total', 'Total', '20362', '1946', '2093', '2249', '2123', '1871', '1959', '1716', '1410', '1183', '957', '737', '656', '534', '299', '288', '165', '133', '43', '-', '-', '-']
1287 ['Cabaceiras (PB)', '2010', 'Total', 'Total', '5035', '403', '409', '435', '484', '432', '381', '357', '327', '292', '300', '244', '200', '232', '161', '110', '107', '59', '45', '44', '6', '7']
1288 ['Cabedelo (PB)', '2010', 'Total', 'Total', '57944', '4641', '4678', '4976', '4974', '5462', '5564', '5192', '4596', '4096', '3683', '2819', '2203', '1592', '1226', '834', '668', '477', '173', '43', '26', '21']
1289 ['Cachoeira dos Índios (PB)', '2010', 'Total', 'Total', '9546', '712', '820', '920', '955', '886', '838', '685', '583', '561', '466', '361', '412', '445', '254', '233', 

...

4329 ['Anitápolis (SC)', '2010', 'Total', 'Total', '3214', '217', '218', '270', '265', '220', '202', '215', '248', '252', '255', '203', '184', '140', '112', '84', '69', '37', '18', '4', '-', '-']
4330 ['Antônio Carlos (SC)', '2010', 'Total', 'Total', '7458', '469', '515', '614', '680', '591', '607', '620', '600', '593', '540', '405', '310', '290', '229', '155', '129', '54', '40', '3', '14', '-']
4331 ['Apiúna (SC)', '2010', 'Total', 'Total', '9600', '646', '726', '905', '915', '791', '834', '716', '779', '672', '640', '518', '438', '288', '307', '181', '144', '71', '20', '10', '-', '-']
4332 ['Arabutã (SC)', '2010', 'Total', 'Total', '4193', '239', '260', '297', '324', '265', '324', '262', '315', '281', '311', '274', '246', '259', '203', '107', '93', '72', '36', '22', '-', '3']
4333 ['Araquari (SC)', '2010', 'Total', 'Total', '24810', '2243', '2353', '2528', '2274', '2246', '2332', '2197', '1834', '1556', '1472', '1236', '847', '665', '394', '264', '209', '104', '27', '-', '8', '20']
4

4648 ['Barra Funda (RS)', '2010', 'Total', 'Total', '2367', '118', '130', '187', '204', '174', '177', '156', '173', '188', '187', '154', '157', '106', '109', '55', '42', '24', '19', '6', '-', '-']
4649 ['Barros Cassal (RS)', '2010', 'Total', 'Total', '11133', '748', '947', '1100', '1027', '722', '820', '743', '705', '720', '798', '718', '598', '529', '302', '256', '218', '103', '51', '15', '11', '4']
4650 ['Benjamin Constant do Sul (RS)', '2010', 'Total', 'Total', '2307', '150', '223', '299', '269', '158', '149', '112', '116', '142', '183', '111', '100', '99', '59', '59', '30', '23', '7', '13', '4', '-']
4651 ['Bento Gonçalves (RS)', '2010', 'Total', 'Total', '107278', '5575', '6252', '7307', '8023', '9898', '10643', '9133', '7764', '8185', '8134', '7265', '5603', '4477', '3070', '2330', '1706', '1101', '623', '180', '10', '-']
4652 ['Boa Vista das Missões (RS)', '2010', 'Total', 'Total', '2114', '123', '150', '215', '203', '145', '153', '138', '132', '156', '170', '140', '111', '66', 

5087 ['Venâncio Aires (RS)', '2010', 'Total', 'Total', '65946', '3438', '4070', '5068', '5300', '5241', '5323', '4983', '4891', '5118', '4970', '4154', '3967', '2801', '2300', '1836', '1208', '874', '339', '65', '-', '-']
5088 ['Vera Cruz (RS)', '2010', 'Total', 'Total', '23983', '1391', '1564', '1946', '1953', '1810', '1945', '1769', '1831', '1876', '1745', '1576', '1170', '1171', '745', '550', '449', '323', '106', '56', '7', '-']
5089 ['Veranópolis (RS)', '2010', 'Total', 'Total', '22810', '1032', '1198', '1489', '1693', '1897', '2017', '1820', '1608', '1751', '1842', '1459', '1450', '977', '852', '702', '470', '358', '118', '77', '-', '-']
5090 ['Vespasiano Corrêa (RS)', '2010', 'Total', 'Total', '1974', '84', '99', '105', '138', '108', '107', '125', '117', '162', '154', '152', '140', '141', '99', '86', '85', '50', '13', '10', '-', '-']
5091 ['Viadutos (RS)', '2010', 'Total', 'Total', '5311', '232', '289', '348', '474', '331', '322', '312', '340', '395', '447', '375', '409', '280', 

5180 ['Terenos (MS)', '2010', 'Total', 'Total', '17146', '1244', '1484', '1570', '1441', '1304', '1304', '1316', '1209', '1186', '1218', '1038', '896', '599', '611', '274', '241', '128', '69', '14', '-', '-']
5181 ['Três Lagoas (MS)', '2010', 'Total', 'Total', '101791', '7762', '7708', '8619', '8787', '9496', '9687', '8575', '7693', '7109', '6475', '5530', '4389', '3254', '2403', '2012', '1085', '707', '367', '99', '17', '18']
5182 ['Vicentina (MS)', '2010', 'Total', 'Total', '5901', '334', '412', '479', '493', '435', '399', '395', '385', '475', '464', '408', '267', '221', '214', '215', '161', '84', '43', '3', '14', '-']
5183 ['Acorizal (MT)', '2010', 'Total', 'Total', '5516', '368', '466', '527', '530', '430', '429', '403', '327', '325', '322', '275', '284', '279', '196', '124', '108', '76', '20', '14', '5', '7']
5184 ['Água Boa (MT)', '2010', 'Total', 'Total', '20856', '1632', '1807', '1926', '1957', '1883', '1967', '1956', '1684', '1543', '1266', '1099', '639', '514', '428', '283', 

5449 ['Jandaia (GO)', '2010', 'Total', 'Total', '6164', '362', '390', '490', '523', '388', '411', '487', '481', '502', '449', '375', '342', '267', '267', '194', '125', '72', '27', '9', '3', '-']
5450 ['Jaraguá (GO)', '2010', 'Total', 'Total', '41870', '3060', '3569', '3960', '3742', '3693', '3735', '3478', '3280', '3235', '2565', '2012', '1716', '1263', '875', '765', '477', '248', '144', '24', '22', '8']
5451 ['Jataí (GO)', '2010', 'Total', 'Total', '88006', '6395', '6900', '7559', '7450', '7946', '7998', '7696', '6939', '6346', '5318', '4580', '3551', '2882', '2156', '1755', '1074', '790', '384', '215', '73', '-']
5452 ['Jaupaci (GO)', '2010', 'Total', 'Total', '3000', '156', '244', '247', '241', '191', '174', '245', '251', '221', '231', '178', '168', '117', '110', '99', '70', '38', '11', '3', '3', '2']
5453 ['Jesúpolis (GO)', '2010', 'Total', 'Total', '2300', '161', '177', '211', '216', '175', '142', '186', '164', '174', '130', '142', '115', '103', '75', '54', '30', '21', '18', '6', 

Como se observa, acima, as primeiras 3 linhas do arquivo são apenas informativas e não deverão ser carregadas (assim temos que configurar o parâmetro skiprows = 3 quando importarmos o csv).

Vemos também que, no fim do arquivo, há linhas que não nos interessam (totailizações, comentários, notas, etc). Assim, será necessário limitar a importação dos dados, configurando o parâmetro skipfooter = 14 (no caso).

Como utilizaremos o skipfooter, é necessário configurar o parâmetro engine='python'

In [52]:
IBGE_Censo_Idade=pd.read_csv(nome_arquivo, delimiter=',',engine='python', skiprows=3, skipfooter=14,encoding = enconding_arquivo)
IBGE_Censo_Idade

,Município,Ano,Situação do domicílio,Sexo,Total,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais
0,Alta Floresta D'Oeste (RO),2010,Total,Total,24392,1851,2107,2401,2588,2200,2092,1867,1795,1715,1566,1178,888,676,551,457,287,115,35,10,12,-
1,Ariquemes (RO),2010,Total,Total,90353,7342,8286,9256,9041,8981,8458,7668,6944,6416,5042,3982,2915,2009,1665,1089,567,513,109,39,19,13
2,Cabixi (RO),2010,Total,Total,6313,515,507,604,595,448,522,462,457,467,415,406,278,228,148,134,73,44,7,-,-,3
3,Cacoal (RO),2010,Total,Total,78574,5923,6275,7473,7859,7758,7018,6617,6067,5647,4832,4072,2671,2048,1582,1154,854,434,237,46,7,-
4,Cerejeiras (RO),2010,Total,Total,17029,1218,1357,1635,1719,1560,1410,1321,1249,1268,1129,836,629,508,478,302,225,111,43,24,6,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5561,Vianópolis (GO),2010,Total,Total,12548,786,1008,1129,1085,1039,1056,1127,1028,950,814,654,466,428,314,250,198,149,45,22,-,-
5562,Vicentinópolis (GO),2010,Total,Total,7371,592,587,622,640,697,669,655,607,588,476,337,326,170,164,106,56,45,23,10,-,-
5563,Vila Boa (GO),2010,Total,Total,4735,447,481,475,429,456,567,436,317,292,237,179,127,98,89,56,26,9,14,-,-,-
5564,Vila Propício (GO),2010,Total,Total,5145,357,490,508,468,360,363,382,369,368,327,281,282,225,142,116,66,32,3,2,3,-


In [53]:
IBGE_Censo_Idade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5566 entries, 0 to 5565
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Município              5566 non-null   object
 1   Ano                    5566 non-null   int64 
 2   Situação do domicílio  5566 non-null   object
 3   Sexo                   5566 non-null   object
 4   Total                  5566 non-null   object
 5   0 a 4 anos             5566 non-null   object
 6   5 a 9 anos             5566 non-null   object
 7   10 a 14 anos           5566 non-null   object
 8   15 a 19 anos           5566 non-null   object
 9   20 a 24 anos           5566 non-null   object
 10  25 a 29 anos           5566 non-null   object
 11  30 a 34 anos           5566 non-null   object
 12  35 a 39 anos           5566 non-null   object
 13  40 a 44 anos           5566 non-null   object
 14  45 a 49 anos           5566 non-null   object
 15  50 a 54 anos         

### - Promovendo transformações nos dados

- Obs: os dados disponíveis são unicamente os de 2010;
- Criar uma nova coluna, padronizada, chamada 'municipio-uf' que será a chave para joins/merges com outros dataframes;
- Atribuir nomes mais sucintos às variáveis;
- Criação de colunas adicionais;
- Reordenar as colunas.

In [54]:
# Criando a nova coluna 'municipio-uf'.
IBGE_Censo_Idade['municipio-uf']=IBGE_Censo_Idade['Município'].str[:-5].str.upper()+'-'+IBGE_Censo_Idade['Município'].str[-3:-1].str.upper()

# Removendo caracteres especiais da coluna recém-criada.
IBGE_Censo_Idade['municipio-uf'] = IBGE_Censo_Idade['municipio-uf'].apply(remover_caracteres_especiais)

IBGE_Censo_Idade.head()

,Município,Ano,Situação do domicílio,Sexo,Total,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais,municipio-uf
0,Alta Floresta D'Oeste (RO),2010,Total,Total,24392,1851,2107,2401,2588,2200,2092,1867,1795,1715,1566,1178,888,676,551,457,287,115,35,10,12,-,ALTA FLORESTA D'OESTE-RO
1,Ariquemes (RO),2010,Total,Total,90353,7342,8286,9256,9041,8981,8458,7668,6944,6416,5042,3982,2915,2009,1665,1089,567,513,109,39,19,13,ARIQUEMES-RO
2,Cabixi (RO),2010,Total,Total,6313,515,507,604,595,448,522,462,457,467,415,406,278,228,148,134,73,44,7,-,-,3,CABIXI-RO
3,Cacoal (RO),2010,Total,Total,78574,5923,6275,7473,7859,7758,7018,6617,6067,5647,4832,4072,2671,2048,1582,1154,854,434,237,46,7,-,CACOAL-RO
4,Cerejeiras (RO),2010,Total,Total,17029,1218,1357,1635,1719,1560,1410,1321,1249,1268,1129,836,629,508,478,302,225,111,43,24,6,-,CEREJEIRAS-RO


In [55]:
print('Shape',IBGE_Censo_Idade.shape)
print('Valores únicos',pd.Series(IBGE_Censo_Idade['municipio-uf']).unique().shape)
# Veremos que não há duplicidades.

Shape (5566, 27)
Valores únicos (5566,)


In [56]:
#Vamos excluir as colunas em que os valores são fixos (e sem interesse): ano, situação do domicilio, sexo.
#Também excluiremos Município (que deu origem à coluna municipio-uf)
cols=['Ano','Situação do domicílio','Sexo','Município']
IBGE_Censo_Idade = IBGE_Censo_Idade.drop(cols,1)
IBGE_Censo_Idade.columns

Index(['Total', '0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
       '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos',
       '40 a 44 anos', '45 a 49 anos', '50 a 54 anos', '55 a 59 anos',
       '60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '75 a 79 anos',
       '80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos',
       '100 anos ou mais', 'municipio-uf'],
      dtype='object')

In [57]:
#Em seguida, vamos transformar as quantidades de pessoas em cada faixa em numérico.

cols = ['Total', '0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
       '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos',
       '40 a 44 anos', '45 a 49 anos', '50 a 54 anos', '55 a 59 anos',
       '60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '75 a 79 anos',
       '80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos',         
       '100 anos ou mais']

for i in cols:
        IBGE_Censo_Idade[i]= pd.to_numeric(IBGE_Censo_Idade[i],errors='coerce')

print(IBGE_Censo_Idade.info())
IBGE_Censo_Idade.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5566 entries, 0 to 5565
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Total             5565 non-null   float64
 1   0 a 4 anos        5565 non-null   float64
 2   5 a 9 anos        5565 non-null   float64
 3   10 a 14 anos      5565 non-null   float64
 4   15 a 19 anos      5565 non-null   float64
 5   20 a 24 anos      5565 non-null   float64
 6   25 a 29 anos      5565 non-null   float64
 7   30 a 34 anos      5565 non-null   float64
 8   35 a 39 anos      5565 non-null   float64
 9   40 a 44 anos      5565 non-null   float64
 10  45 a 49 anos      5565 non-null   float64
 11  50 a 54 anos      5565 non-null   float64
 12  55 a 59 anos      5565 non-null   float64
 13  60 a 64 anos      5565 non-null   float64
 14  65 a 69 anos      5565 non-null   float64
 15  70 a 74 anos      5565 non-null   float64
 16  75 a 79 anos      5565 non-null   float64


,Total,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais,municipio-uf
0,24392.0,1851.0,2107.0,2401.0,2588.0,2200.0,2092.0,1867.0,1795.0,1715.0,1566.0,1178.0,888.0,676.0,551.0,457.0,287.0,115.0,35.0,10.0,12.0,NaN,ALTA FLORESTA D'OESTE-RO
1,90353.0,7342.0,8286.0,9256.0,9041.0,8981.0,8458.0,7668.0,6944.0,6416.0,5042.0,3982.0,2915.0,2009.0,1665.0,1089.0,567.0,513.0,109.0,39.0,19.0,13.0,ARIQUEMES-RO
2,6313.0,515.0,507.0,604.0,595.0,448.0,522.0,462.0,457.0,467.0,415.0,406.0,278.0,228.0,148.0,134.0,73.0,44.0,7.0,NaN,NaN,3.0,CABIXI-RO
3,78574.0,5923.0,6275.0,7473.0,7859.0,7758.0,7018.0,6617.0,6067.0,5647.0,4832.0,4072.0,2671.0,2048.0,1582.0,1154.0,854.0,434.0,237.0,46.0,7.0,NaN,CACOAL-RO
4,17029.0,1218.0,1357.0,1635.0,1719.0,1560.0,1410.0,1321.0,1249.0,1268.0,1129.0,836.0,629.0,508.0,478.0,302.0,225.0,111.0,43.0,24.0,6.0,NaN,CEREJEIRAS-RO


In [58]:
## Vemos acima que, após convertermos os valores em numéricos, em vez de 5566 informações não nulas para todos os municípios
## passamos a ter apenas 5565.

## Vamos tentar encontrar o valor diferente.
divergencia = IBGE_Censo_Idade.loc[IBGE_Censo_Idade['Total'].isna()]
print (divergencia.shape)
divergencia.head()

(1, 23)


,Total,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais,municipio-uf
1074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COCOCI-CE


In [59]:
## Temos, assim, que a cidade Cococi-CE não apresenta qualquer valor para os campos. Vamos, assim, removê-la.

IBGE_Censo_Idade = IBGE_Censo_Idade.drop(divergencia.index)

# Repetindo o comando, vemos que não há mais divergências.
divergencia = IBGE_Censo_Idade.loc[IBGE_Censo_Idade['Total'].isna()]
print (divergencia.shape)

print (IBGE_Censo_Idade.info())


(0, 23)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5565
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Total             5565 non-null   float64
 1   0 a 4 anos        5565 non-null   float64
 2   5 a 9 anos        5565 non-null   float64
 3   10 a 14 anos      5565 non-null   float64
 4   15 a 19 anos      5565 non-null   float64
 5   20 a 24 anos      5565 non-null   float64
 6   25 a 29 anos      5565 non-null   float64
 7   30 a 34 anos      5565 non-null   float64
 8   35 a 39 anos      5565 non-null   float64
 9   40 a 44 anos      5565 non-null   float64
 10  45 a 49 anos      5565 non-null   float64
 11  50 a 54 anos      5565 non-null   float64
 12  55 a 59 anos      5565 non-null   float64
 13  60 a 64 anos      5565 non-null   float64
 14  65 a 69 anos      5565 non-null   float64
 15  70 a 74 anos      5565 non-null   float64
 16  75 a 79 anos      5565 non-null   

In [60]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Censo_Idade)

Shape do dataframe: (5565, 23)
Colunas que contêm valores faltantes: ['80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos', '100 anos ou mais']
Número de linhas com valores faltantes: 4258 



['80 a 84 anos',
 '85 a 89 anos',
 '90 a 94 anos',
 '95 a 99 anos',
 '100 anos ou mais']

In [61]:
## Vemos que para as colunas das idades mais elevadas (e apenas para elas) há valores NaN.

## Vamos substituir os NaN por zeros para não influenciar indevidamente a criação das novas colunas da próxima célula
## Somar número com NaN geraria erro.

IBGE_Censo_Idade = IBGE_Censo_Idade.fillna(0)
print (IBGE_Censo_Idade.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5565
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Total             5565 non-null   float64
 1   0 a 4 anos        5565 non-null   float64
 2   5 a 9 anos        5565 non-null   float64
 3   10 a 14 anos      5565 non-null   float64
 4   15 a 19 anos      5565 non-null   float64
 5   20 a 24 anos      5565 non-null   float64
 6   25 a 29 anos      5565 non-null   float64
 7   30 a 34 anos      5565 non-null   float64
 8   35 a 39 anos      5565 non-null   float64
 9   40 a 44 anos      5565 non-null   float64
 10  45 a 49 anos      5565 non-null   float64
 11  50 a 54 anos      5565 non-null   float64
 12  55 a 59 anos      5565 non-null   float64
 13  60 a 64 anos      5565 non-null   float64
 14  65 a 69 anos      5565 non-null   float64
 15  70 a 74 anos      5565 non-null   float64
 16  75 a 79 anos      5565 non-null   float64


In [62]:
# Temos, então, 5565 municípios com informações completas.

#Vamos, agora, reduzir o número de faixas de valores considerando ranges de 20 anos em vez de 5 anos e 
#transformar a informação como um percentual do total.
IBGE_Censo_Idade['05_0-19_anos_%'] = (IBGE_Censo_Idade['0 a 4 anos']+IBGE_Censo_Idade['5 a 9 anos'] \
+IBGE_Censo_Idade['10 a 14 anos']+IBGE_Censo_Idade['15 a 19 anos'])/IBGE_Censo_Idade['Total']*100

IBGE_Censo_Idade['05_20-39_anos_%'] = (IBGE_Censo_Idade['20 a 24 anos']+IBGE_Censo_Idade['25 a 29 anos']\
+IBGE_Censo_Idade['30 a 34 anos']+IBGE_Censo_Idade['35 a 39 anos'])/IBGE_Censo_Idade['Total']*100

IBGE_Censo_Idade['05_40-59_anos_%'] = (IBGE_Censo_Idade['40 a 44 anos']+IBGE_Censo_Idade['45 a 49 anos']\
+IBGE_Censo_Idade['50 a 54 anos']+IBGE_Censo_Idade['55 a 59 anos'])/IBGE_Censo_Idade['Total']*100

IBGE_Censo_Idade['05_60+_anos_%'] = (IBGE_Censo_Idade['60 a 64 anos']+IBGE_Censo_Idade['65 a 69 anos'] \
+IBGE_Censo_Idade['70 a 74 anos']+IBGE_Censo_Idade['75 a 79 anos']+IBGE_Censo_Idade['80 a 84 anos'] \
+IBGE_Censo_Idade['85 a 89 anos']+IBGE_Censo_Idade['90 a 94 anos']+IBGE_Censo_Idade['95 a 99 anos'] \
+IBGE_Censo_Idade['100 anos ou mais'])/IBGE_Censo_Idade['Total']*100


# Agora apagar as colunas originais.
cols=['0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
       '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos',
       '40 a 44 anos', '45 a 49 anos', '50 a 54 anos', '55 a 59 anos',
       '60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '75 a 79 anos',
       '80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos',
       '100 anos ou mais']
IBGE_Censo_Idade = IBGE_Censo_Idade.drop(cols,1)


IBGE_Censo_Idade.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5565
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Total            5565 non-null   float64
 1   municipio-uf     5565 non-null   object 
 2   05_0-19_anos_%   5565 non-null   float64
 3   05_20-39_anos_%  5565 non-null   float64
 4   05_40-59_anos_%  5565 non-null   float64
 5   05_60+_anos_%    5565 non-null   float64
dtypes: float64(5), object(1)
memory usage: 304.3+ KB


In [63]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Censo_Idade)

Shape do dataframe: (5565, 6)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [64]:
# Vamos renomar a coluna Total

IBGE_Censo_Idade = IBGE_Censo_Idade.rename({'Total': '05_total_residentes'}, axis = 1)

# ...e agora reorndeá-las.

cols = ['municipio-uf','05_total_residentes', '05_0-19_anos_%', '05_20-39_anos_%',
       '05_40-59_anos_%', '05_60+_anos_%']

IBGE_Censo_Idade = IBGE_Censo_Idade[cols]

print(IBGE_Censo_Idade.shape)
IBGE_Censo_Idade.info()

(5565, 6)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5565
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   municipio-uf         5565 non-null   object 
 1   05_total_residentes  5565 non-null   float64
 2   05_0-19_anos_%       5565 non-null   float64
 3   05_20-39_anos_%      5565 non-null   float64
 4   05_40-59_anos_%      5565 non-null   float64
 5   05_60+_anos_%        5565 non-null   float64
dtypes: float64(5), object(1)
memory usage: 304.3+ KB


In [65]:
IBGE_Censo_Idade.head()

,municipio-uf,05_total_residentes,05_0-19_anos_%,05_20-39_anos_%,05_40-59_anos_%,05_60+_anos_%
0,ALTA FLORESTA D'OESTE-RO,24392.0,36.680059,32.609052,21.921122,8.785667
1,ARIQUEMES-RO,90353.0,37.547176,35.473089,20.314765,6.666076
2,CABIXI-RO,6313.0,35.181372,29.922382,24.805956,10.090290
3,CACOAL-RO,78574.0,35.037035,34.947947,21.918192,8.096826
4,CEREJEIRAS-RO,17029.0,34.817077,32.532738,22.678959,9.965353


### - Cria '.csv' com o resultado das transformações na base original.

In [66]:
IBGE_Censo_Idade.to_csv('ntbk 01_05 - ibge censo 2010 idades.csv',index=False)

## 2.6. Tabela 1554 - Pessoas de 10 anos ou mais de idade, por nível de instrução (tabela1554.csv)

Descrição:

Censo Demográfico - 2010 O Censo Demográfico tem por objetivo contar os habitantes do território nacional, identificar suas características e revelar como vivem os brasileiros, produzindo informações imprescindíveis para a definição de políticas públicas e a tomada de decisões de investimentos da iniciativa privada ou de qualquer nível de governo. Também constitui a única fonte de referência sobre a situação de vida da população nos municípios e em seus recortes internos, como distritos, bairros e localidades, rurais ou urbanas, cujas realidades dependem de seus resultados para serem conhecidas e terem seus dados atualizados.

Tabela 1554 - Pessoas de 10 anos ou mais de idade, por nível de instrução - Resultados Gerais da Amostra


- Fonte: Site do Instituto Brasileiro de Geografia e Estatística na Internet (Sistema IBGE de Recuperação Automática - SIDRA);


- Endereço eletrônico: https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela1554.csv&terr=N&rank=-&query=t/1554/n6/all/v/allxp/p/all/c1568/all/d/v140%200/l/v,c1568,t%2Bp


- Data da coleta: 19/3/2021;


- Possui metadados: sim, mas sem descrição das variáveis ( https://metadados.ibge.gov.br/consulta/estatisticos/operacoes-estatisticas/CD).


### - Leitura do arquivo '.csv'.

Iniciamos com a visualização do arquivo '.csv' para verificarmos sua estrutura original e analisarmos as linhas que efetivamente deverão ser lidas.

In [67]:
nome_arquivo = 'tabela1554.csv'
enconding_arquivo = ler_csv(nome_arquivo) #Chama a função para a leitura do arquivo '.csv'

0
1 ['Tabela 1554 - Pessoas de 10 anos ou mais de idade, por nível de instrução - Resultados Gerais da Amostra']
2 ['Variável - Pessoas de 10 anos ou mais de idade (Pessoas)']
3 ['Município', 'Ano', 'Nível de instrução']
4 ['Município', 'Ano', 'Total', 'Sem instrução e fundamental incompleto', 'Fundamental completo e médio incompleto', 'Médio completo e superior incompleto', 'Superior completo', 'Não determinado']
5 ["Alta Floresta D'Oeste (RO)", '2010', '20434', '13470', '2925', '3121', '801', '117']
6 ['Ariquemes (RO)', '2010', '74725', '40351', '12630', '16799', '4275', '671']
7 ['Cabixi (RO)', '2010', '5291', '3487', '730', '840', '193', '41']
8 ['Cacoal (RO)', '2010', '66376', '36674', '12244', '12943', '4090', '425']
9 ['Cerejeiras (RO)', '2010', '14454', '8642', '2256', '2562', '765', '229']
10 ['Colorado do Oeste (RO)', '2010', '15769', '9422', '2425', '2799', '951', '171']
11 ['Corumbiara (RO)', '2010', '7419', '5220', '1319', '626', '245', '10']
12 ['Costa Marques (RO)', '201

163 ['Altamira (PA)', '2010', '79787', '47335', '14966', '13944', '3162', '380']
164 ['Anajás (PA)', '2010', '17519', '14992', '1549', '782', '154', '43']
165 ['Ananindeua (PA)', '2010', '394224', '163319', '86151', '122303', '20459', '1991']
166 ['Anapu (PA)', '2010', '15676', '12064', '1999', '1295', '252', '66']
167 ['Augusto Corrêa (PA)', '2010', '30482', '23916', '3998', '2268', '271', '28']
168 ['Aurora do Pará (PA)', '2010', '20831', '16121', '2627', '1687', '303', '93']
169 ['Aveiro (PA)', '2010', '11846', '8440', '2205', '956', '147', '98']
170 ['Bagre (PA)', '2010', '17104', '14284', '1583', '1067', '103', '67']
171 ['Baião (PA)', '2010', '28543', '19252', '4203', '4320', '539', '229']
172 ['Bannach (PA)', '2010', '2720', '2044', '300', '326', '35', '15']
173 ['Barcarena (PA)', '2010', '79283', '42629', '16915', '17044', '2585', '110']
174 ['Belém (PA)', '2010', '1188026', '455026', '232027', '370234', '123915', '6823']
175 ['Belterra (PA)', '2010', '12609', '8660', '1842', '

479 ['Barra do Corda (MA)', '2010', '66037', '45569', '9915', '8500', '1802', '251']
480 ['Barreirinhas (MA)', '2010', '41092', '29979', '6332', '3919', '732', '131']
481 ['Belágua (MA)', '2010', '4677', '3480', '619', '525', '41', '13']
482 ['Bela Vista do Maranhão (MA)', '2010', '9436', '7038', '1308', '959', '130', '-']
483 ['Benedito Leite (MA)', '2010', '4528', '3450', '568', '447', '60', '4']
484 ['Bequimão (MA)', '2010', '16642', '10706', '2782', '2875', '196', '83']
485 ['Bernardo do Mearim (MA)', '2010', '4889', '3333', '678', '755', '112', '12']
486 ['Boa Vista do Gurupi (MA)', '2010', '5921', '4338', '817', '559', '129', '77']
487 ['Bom Jardim (MA)', '2010', '30068', '23428', '3622', '2348', '440', '230']
488 ['Bom Jesus das Selvas (MA)', '2010', '21748', '16069', '3786', '1412', '259', '221']
489 ['Bom Lugar (MA)', '2010', '11678', '8315', '1863', '1260', '120', '120']
490 ['Brejo (MA)', '2010', '25759', '19028', '3189', '2871', '516', '156']
491 ['Brejo de Areia (MA)', '20

626 ['São Bernardo (MA)', '2010', '20629', '14775', '3199', '2123', '489', '42']
627 ['São Domingos do Azeitão (MA)', '2010', '5442', '3850', '663', '737', '103', '90']
628 ['São Domingos do Maranhão (MA)', '2010', '26844', '19089', '4101', '2898', '715', '41']
629 ['São Félix de Balsas (MA)', '2010', '3833', '2889', '447', '412', '75', '8']
630 ['São Francisco do Brejão (MA)', '2010', '8025', '5611', '1369', '862', '122', '61']
631 ['São Francisco do Maranhão (MA)', '2010', '9890', '7930', '968', '810', '150', '32']
632 ['São João Batista (MA)', '2010', '16048', '9927', '2761', '2939', '277', '145']
633 ['São João do Carú (MA)', '2010', '9188', '7309', '1098', '732', '42', '8']
634 ['São João do Paraíso (MA)', '2010', '8629', '5617', '1563', '1246', '202', '-']
635 ['São João do Soter (MA)', '2010', '13056', '10558', '1441', '821', '148', '89']
636 ['São João dos Patos (MA)', '2010', '20565', '13800', '3885', '2254', '626', '-']
637 ['São José de Ribamar (MA)', '2010', '134474', '5523

1375 ['Nova Palmeira (PB)', '2010', '3693', '2653', '537', '403', '81', '19']
1376 ["Olho d'Água (PB)", '2010', '5803', '4293', '750', '578', '149', '33']
1377 ['Olivedos (PB)', '2010', '3075', '1998', '454', '559', '61', '4']
1378 ['Ouro Velho (PB)', '2010', '2484', '1567', '349', '482', '80', '7']
1379 ['Parari (PB)', '2010', '1057', '793', '87', '151', '25', '-']
1380 ['Passagem (PB)', '2010', '1839', '1229', '231', '325', '51', '2']
1381 ['Patos (PB)', '2010', '84597', '46564', '13019', '19428', '5308', '278']
1382 ['Paulista (PB)', '2010', '10067', '7523', '1199', '1120', '224', '-']
1383 ['Pedra Branca (PB)', '2010', '3076', '2064', '411', '513', '76', '12']
1384 ['Pedra Lavrada (PB)', '2010', '6253', '4738', '758', '629', '121', '7']
1385 ['Pedras de Fogo (PB)', '2010', '22417', '16399', '2962', '2509', '480', '68']
1386 ['Piancó (PB)', '2010', '13145', '8350', '1874', '2250', '450', '220']
1387 ['Picuí (PB)', '2010', '15206', '10705', '2137', '1951', '359', '55']
1388 ['Pilar (

...

3874 ['Tatuí (SP)', '2010', '92081', '43982', '17901', '22762', '7118', '317']
3875 ['Taubaté (SP)', '2010', '241653', '91011', '41222', '77194', '31053', '1174']
3876 ['Tejupá (SP)', '2010', '4002', '2705', '641', '563', '91', '2']
3877 ['Teodoro Sampaio (SP)', '2010', '18186', '9204', '3535', '4436', '970', '41']
3878 ['Terra Roxa (SP)', '2010', '7322', '3876', '1335', '1588', '507', '15']
3879 ['Tietê (SP)', '2010', '32057', '15225', '6111', '7364', '3133', '224']
3880 ['Timburi (SP)', '2010', '2313', '1378', '424', '394', '103', '13']
3881 ['Torre de Pedra (SP)', '2010', '1959', '1098', '364', '450', '43', '3']
3882 ['Torrinha (SP)', '2010', '8243', '4552', '1499', '1611', '545', '37']
3883 ['Trabiju (SP)', '2010', '1326', '721', '263', '285', '52', '5']
3884 ['Tremembé (SP)', '2010', '35475', '14618', '7398', '9589', '3768', '103']
3885 ['Três Fronteiras (SP)', '2010', '4835', '2693', '663', '1178', '293', '8']
3886 ['Tuiuti (SP)', '2010', '5238', '3023', '816', '1133', '252', '13

4123 ['Mariluz (PR)', '2010', '8634', '5830', '1154', '1296', '285', '68']
4124 ['Maringá (PR)', '2010', '315711', '112628', '54535', '98763', '49013', '772']
4125 ['Mariópolis (PR)', '2010', '5425', '3276', '886', '943', '263', '57']
4126 ['Maripá (PR)', '2010', '5043', '2658', '854', '1160', '367', '4']
4127 ['Marmeleiro (PR)', '2010', '11925', '7220', '2004', '2113', '588', '-']
4128 ['Marquinho (PR)', '2010', '4131', '3131', '555', '367', '78', '-']
4129 ['Marumbi (PR)', '2010', '4057', '2242', '608', '882', '225', '101']
4130 ['Matelândia (PR)', '2010', '13719', '7386', '2469', '2930', '871', '63']
4131 ['Matinhos (PR)', '2010', '25400', '12651', '4816', '5913', '1985', '36']
4132 ['Mato Rico (PR)', '2010', '3267', '2254', '472', '446', '47', '48']
4133 ['Mauá da Serra (PR)', '2010', '7063', '4580', '1348', '892', '223', '19']
4134 ['Medianeira (PR)', '2010', '36067', '17886', '6736', '7704', '3697', '44']
4135 ['Mercedes (PR)', '2010', '4391', '2473', '780', '916', '222', '-']
41

4873 ['Muitos Capões (RS)', '2010', '2550', '1506', '628', '335', '81', '-']
4874 ['Muliterno (RS)', '2010', '1601', '1074', '238', '236', '52', '2']
4875 ['Não-Me-Toque (RS)', '2010', '14091', '6746', '2634', '3613', '1078', '20']
4876 ['Nicolau Vergueiro (RS)', '2010', '1511', '952', '209', '259', '88', '2']
4877 ['Nonoai (RS)', '2010', '10388', '5999', '1844', '1684', '775', '85']
4878 ['Nova Alvorada (RS)', '2010', '2750', '1648', '432', '515', '132', '24']
4879 ['Nova Araçá (RS)', '2010', '3603', '1776', '725', '887', '209', '6']
4880 ['Nova Bassano (RS)', '2010', '7884', '4397', '1452', '1562', '443', '30']
4881 ['Nova Boa Vista (RS)', '2010', '1800', '1030', '339', '305', '124', '2']
4882 ['Nova Bréscia (RS)', '2010', '2901', '1710', '447', '602', '140', '3']
4883 ['Nova Candelária (RS)', '2010', '2496', '1564', '434', '413', '85', '-']
4884 ['Nova Esperança do Sul (RS)', '2010', '4166', '2249', '862', '858', '169', '29']
4885 ['Nova Hartz (RS)', '2010', '15512', '9320', '3570',

5123 ['Campo Grande (MS)', '2010', '673730', '263883', '123741', '196953', '86897', '2256']
5124 ['Caracol (MS)', '2010', '4529', '2936', '851', '591', '142', '8']
5125 ['Cassilândia (MS)', '2010', '18170', '10365', '2544', '3363', '1840', '58']
5126 ['Chapadão do Sul (MS)', '2010', '16421', '7842', '3150', '3809', '1605', '14']
5127 ['Corguinho (MS)', '2010', '4114', '2750', '582', '533', '210', '39']
5128 ['Coronel Sapucaia (MS)', '2010', '11023', '8071', '1312', '928', '448', '264']
5129 ['Corumbá (MS)', '2010', '85366', '45009', '15740', '18327', '6002', '287']
5130 ['Costa Rica (MS)', '2010', '16790', '9332', '3214', '3209', '970', '65']
5131 ['Coxim (MS)', '2010', '27504', '16247', '4530', '4795', '1813', '118']
5132 ['Deodápolis (MS)', '2010', '10303', '5908', '1703', '2001', '664', '28']
5133 ['Dois Irmãos do Buriti (MS)', '2010', '8607', '5735', '1308', '1145', '304', '116']
5134 ['Douradina (MS)', '2010', '4473', '2676', '740', '826', '231', '-']
5135 ['Dourados (MS)', '2010'

5373 ['Campinaçu (GO)', '2010', '3133', '2211', '691', '178', '48', '6']
5374 ['Campinorte (GO)', '2010', '9391', '5891', '1620', '1500', '303', '78']
5375 ['Campo Alegre de Goiás (GO)', '2010', '5013', '3024', '920', '754', '298', '17']
5376 ['Campo Limpo de Goiás (GO)', '2010', '5133', '3434', '887', '649', '130', '32']
5377 ['Campos Belos (GO)', '2010', '15196', '8559', '2451', '3147', '1002', '37']
5378 ['Campos Verdes (GO)', '2010', '4315', '2781', '737', '660', '100', '37']
5379 ['Carmo do Rio Verde (GO)', '2010', '7689', '4301', '1377', '1650', '352', '10']
5380 ['Castelândia (GO)', '2010', '3109', '1865', '570', '517', '102', '55']
5381 ['Catalão (GO)', '2010', '74561', '32422', '12665', '22188', '7162', '123']
5382 ['Caturaí (GO)', '2010', '3992', '2637', '666', '537', '149', '2']
5383 ['Cavalcante (GO)', '2010', '7464', '5458', '875', '923', '207', '-']
5384 ['Ceres (GO)', '2010', '18199', '8193', '3215', '5239', '1534', '18']
5385 ['Cezarina (GO)', '2010', '6534', '3431', '1

Como se observa, acima, as primeiras linhas do arquivo são apenas informativas e não deverão ser carregadas (assim temos que configurar o parâmetro skiprows quando importarmos o csv).

Vemos também que, no fim do arquivo, há linhas que não nos interessam (totailizações, comentários, notas, etc). Assim, será necessário limitar a importação dos dados, configurando o parâmetro skipfooter.

Como utilizaremos o skipfooter, é necessário configurar o parâmetro engine='python'

In [68]:
IBGE_Censo_Instrucao=pd.read_csv(nome_arquivo, delimiter=',',engine='python', skiprows=3, skipfooter=13,encoding = enconding_arquivo)
IBGE_Censo_Instrucao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 8 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Município                                5565 non-null   object
 1   Ano                                      5565 non-null   int64 
 2   Total                                    5565 non-null   int64 
 3   Sem instrução e fundamental incompleto   5565 non-null   int64 
 4   Fundamental completo e médio incompleto  5565 non-null   int64 
 5   Médio completo e superior incompleto     5565 non-null   int64 
 6   Superior completo                        5565 non-null   int64 
 7   Não determinado                          5565 non-null   object
dtypes: int64(6), object(2)
memory usage: 347.9+ KB


In [69]:
IBGE_Censo_Instrucao

,Município,Ano,Total,Sem instrução e fundamental incompleto,Fundamental completo e médio incompleto,Médio completo e superior incompleto,Superior completo,Não determinado
0,Alta Floresta D'Oeste (RO),2010,20434,13470,2925,3121,801,117
1,Ariquemes (RO),2010,74725,40351,12630,16799,4275,671
2,Cabixi (RO),2010,5291,3487,730,840,193,41
3,Cacoal (RO),2010,66376,36674,12244,12943,4090,425
4,Cerejeiras (RO),2010,14454,8642,2256,2562,765,229
...,...,...,...,...,...,...,...,...
5560,Vianópolis (GO),2010,10754,6253,1930,2020,480,72
5561,Vicentinópolis (GO),2010,6192,3934,936,1021,283,18
5562,Vila Boa (GO),2010,3807,2335,600,724,80,69
5563,Vila Propício (GO),2010,4298,2938,873,377,87,22


### - Promovendo transformações nos dados

- Obs: os dados disponíveis são unicamente os de 2010;
- Criar uma nova coluna, padronizada, chamada 'municipio-uf' que será a chave para joins/merges com outros dataframes;
- Atribuir nomes mais sucintos às variáveis;
- Criação de colunas adicionais;
- Excluindo colunas desnecessárias;
- Reordenar as colunas.

In [70]:
# Criando a nova coluna 'municipio-uf'.
IBGE_Censo_Instrucao['municipio-uf']=IBGE_Censo_Instrucao['Município'].str[:-5].str.upper()+'-'+IBGE_Censo_Instrucao['Município'].str[-3:-1].str.upper()

# Removendo caracteres especiais da coluna recém-criada.
IBGE_Censo_Instrucao['municipio-uf'] = IBGE_Censo_Instrucao['municipio-uf'].apply(remover_caracteres_especiais)

IBGE_Censo_Instrucao.head()

,Município,Ano,Total,Sem instrução e fundamental incompleto,Fundamental completo e médio incompleto,Médio completo e superior incompleto,Superior completo,Não determinado,municipio-uf
0,Alta Floresta D'Oeste (RO),2010,20434,13470,2925,3121,801,117,ALTA FLORESTA D'OESTE-RO
1,Ariquemes (RO),2010,74725,40351,12630,16799,4275,671,ARIQUEMES-RO
2,Cabixi (RO),2010,5291,3487,730,840,193,41,CABIXI-RO
3,Cacoal (RO),2010,66376,36674,12244,12943,4090,425,CACOAL-RO
4,Cerejeiras (RO),2010,14454,8642,2256,2562,765,229,CEREJEIRAS-RO


In [71]:
print('Shape',IBGE_Censo_Instrucao.shape)
print('Valores únicos',pd.Series(IBGE_Censo_Instrucao['municipio-uf']).unique().shape)
# Veremos que não há duplicidades.

Shape (5565, 9)
Valores únicos (5565,)


In [72]:
#Vamos excluir as colunas em que os valores são fixos (e sem interesse): ano.
#Também excluiremos Município (que deu origem à coluna municipio-uf)
cols=['Ano','Município']
IBGE_Censo_Instrucao = IBGE_Censo_Instrucao.drop(cols,1)
IBGE_Censo_Instrucao.columns

Index(['Total', 'Sem instrução e fundamental incompleto',
       'Fundamental completo e médio incompleto',
       'Médio completo e superior incompleto', 'Superior completo',
       'Não determinado', 'municipio-uf'],
      dtype='object')

In [73]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Censo_Instrucao)

Shape do dataframe: (5565, 7)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [74]:
#Em seguida, vamos transformar as quantidades de pessoas em cada faixa em numérico.

cols = ['Total', 'Sem instrução e fundamental incompleto',
       'Fundamental completo e médio incompleto',
       'Médio completo e superior incompleto', 'Superior completo',
       'Não determinado']
for i in cols:
    IBGE_Censo_Instrucao[i]=pd.to_numeric(IBGE_Censo_Instrucao[i],errors='coerce')

print(IBGE_Censo_Instrucao.info())
IBGE_Censo_Instrucao.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 7 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Total                                    5565 non-null   int64  
 1   Sem instrução e fundamental incompleto   5565 non-null   int64  
 2   Fundamental completo e médio incompleto  5565 non-null   int64  
 3   Médio completo e superior incompleto     5565 non-null   int64  
 4   Superior completo                        5565 non-null   int64  
 5   Não determinado                          4754 non-null   float64
 6   municipio-uf                             5565 non-null   object 
dtypes: float64(1), int64(5), object(1)
memory usage: 304.5+ KB
None


,Total,Sem instrução e fundamental incompleto,Fundamental completo e médio incompleto,Médio completo e superior incompleto,Superior completo,Não determinado,municipio-uf
0,20434,13470,2925,3121,801,117.0,ALTA FLORESTA D'OESTE-RO
1,74725,40351,12630,16799,4275,671.0,ARIQUEMES-RO
2,5291,3487,730,840,193,41.0,CABIXI-RO
3,66376,36674,12244,12943,4090,425.0,CACOAL-RO
4,14454,8642,2256,2562,765,229.0,CEREJEIRAS-RO


In [75]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Censo_Instrucao)

Shape do dataframe: (5565, 7)
Colunas que contêm valores faltantes: ['Não determinado']
Número de linhas com valores faltantes: 811 



['Não determinado']

In [76]:
## Vemos que na coluna "Não determinado" (e apenas nela) há valores NaN.

## Vamos substituir os NaN por zeros para não influenciar indevidamente a criação das novas colunas da próxima célula
## Somar número com NaN geraria erro.

IBGE_Censo_Instrucao['Não determinado'] = IBGE_Censo_Instrucao['Não determinado'].fillna(0)
print (IBGE_Censo_Instrucao.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 7 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Total                                    5565 non-null   int64  
 1   Sem instrução e fundamental incompleto   5565 non-null   int64  
 2   Fundamental completo e médio incompleto  5565 non-null   int64  
 3   Médio completo e superior incompleto     5565 non-null   int64  
 4   Superior completo                        5565 non-null   int64  
 5   Não determinado                          5565 non-null   float64
 6   municipio-uf                             5565 non-null   object 
dtypes: float64(1), int64(5), object(1)
memory usage: 304.5+ KB
None


In [77]:
# Temos, então, 5565 municípios com informações completas.

#Vamos, agora,transformar a informação como um percentual do total.
IBGE_Censo_Instrucao['06_Sem_Instrução_e_Fundamental_Incompleto%'] = \
IBGE_Censo_Instrucao['Sem instrução e fundamental incompleto']/IBGE_Censo_Instrucao['Total']

IBGE_Censo_Instrucao['06_Medio_Incompleto_%'] = \
IBGE_Censo_Instrucao['Fundamental completo e médio incompleto']/IBGE_Censo_Instrucao['Total']*100

IBGE_Censo_Instrucao['06_Superior_Incompleto_%'] = \
IBGE_Censo_Instrucao['Médio completo e superior incompleto']/IBGE_Censo_Instrucao['Total']*100

IBGE_Censo_Instrucao['06_Superior_Completo_%'] = \
IBGE_Censo_Instrucao['Superior completo']/IBGE_Censo_Instrucao['Total']*100

IBGE_Censo_Instrucao['06_Indeterminado_%'] = \
IBGE_Censo_Instrucao['Não determinado']/IBGE_Censo_Instrucao['Total']*100

#Podemos agora apagar as colunas originais.
cols=['Sem instrução e fundamental incompleto',
       'Fundamental completo e médio incompleto',
       'Médio completo e superior incompleto', 'Superior completo',
       'Não determinado']
IBGE_Censo_Instrucao = IBGE_Censo_Instrucao.drop(cols,1)

IBGE_Censo_Instrucao.head()

,Total,municipio-uf,06_Sem_Instrução_e_Fundamental_Incompleto%,06_Medio_Incompleto_%,06_Superior_Incompleto_%,06_Superior_Completo_%,06_Indeterminado_%
0,20434,ALTA FLORESTA D'OESTE-RO,0.659195,14.314378,15.273564,3.919937,0.572575
1,74725,ARIQUEMES-RO,0.539993,16.901974,22.481097,5.720977,0.897959
2,5291,CABIXI-RO,0.659044,13.797014,15.876016,3.647704,0.774901
3,66376,CACOAL-RO,0.552519,18.446426,19.499518,6.161866,0.640292
4,14454,CEREJEIRAS-RO,0.597897,15.608136,17.725197,5.292653,1.584337


In [78]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Censo_Instrucao)

Shape do dataframe: (5565, 7)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [79]:
# Vamos renomar a coluna Total

IBGE_Censo_Instrucao = IBGE_Censo_Instrucao.rename({'Total': '06_total_instrução'}, axis = 1)

# ...e agora reorndeá-las.

cols = ['municipio-uf','06_total_instrução', '06_Sem_Instrução_e_Fundamental_Incompleto%',
       '06_Medio_Incompleto_%', '06_Superior_Incompleto_%',
       '06_Superior_Completo_%', '06_Indeterminado_%']

IBGE_Censo_Instrucao = IBGE_Censo_Instrucao[cols]

print(IBGE_Censo_Instrucao.shape)
IBGE_Censo_Instrucao.info()

(5565, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   municipio-uf                                5565 non-null   object 
 1   06_total_instrução                          5565 non-null   int64  
 2   06_Sem_Instrução_e_Fundamental_Incompleto%  5565 non-null   float64
 3   06_Medio_Incompleto_%                       5565 non-null   float64
 4   06_Superior_Incompleto_%                    5565 non-null   float64
 5   06_Superior_Completo_%                      5565 non-null   float64
 6   06_Indeterminado_%                          5565 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 304.5+ KB


In [80]:
IBGE_Censo_Instrucao.head()

,municipio-uf,06_total_instrução,06_Sem_Instrução_e_Fundamental_Incompleto%,06_Medio_Incompleto_%,06_Superior_Incompleto_%,06_Superior_Completo_%,06_Indeterminado_%
0,ALTA FLORESTA D'OESTE-RO,20434,0.659195,14.314378,15.273564,3.919937,0.572575
1,ARIQUEMES-RO,74725,0.539993,16.901974,22.481097,5.720977,0.897959
2,CABIXI-RO,5291,0.659044,13.797014,15.876016,3.647704,0.774901
3,CACOAL-RO,66376,0.552519,18.446426,19.499518,6.161866,0.640292
4,CEREJEIRAS-RO,14454,0.597897,15.608136,17.725197,5.292653,1.584337


### - Cria '.csv' com o resultado das transformações na base original.

In [81]:
IBGE_Censo_Instrucao.to_csv('ntbk 01_06 - ibge censo instrucao.csv',index=False)

## 2.7. Tabela 2030 - Pessoas de 10 anos ou mais de idade, por classes de rendimento nominal mensal (tabela2030.csv)

Descrição:

Censo Demográfico - 2010 O Censo Demográfico tem por objetivo contar os habitantes do território nacional, identificar suas características e revelar como vivem os brasileiros, produzindo informações imprescindíveis para a definição de políticas públicas e a tomada de decisões de investimentos da iniciativa privada ou de qualquer nível de governo. Também constitui a única fonte de referência sobre a situação de vida da população nos municípios e em seus recortes internos, como distritos, bairros e localidades, rurais ou urbanas, cujas realidades dependem de seus resultados para serem conhecidas e terem seus dados atualizados.

Tabela 2030 - Pessoas de 10 anos ou mais de idade, por classes de rendimento nominal mensal - Resultados Gerais da Amostra.

- Fonte: Site do Instituto Brasileiro de Geografia e Estatística na Internet (Sistema IBGE de Recuperação Automática - SIDRA);


- Endereço eletrônico: https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela2030.csv&terr=N&rank=-&query=t/2030/n6/all/v/allxp/p/last%201/c11570/all/d/v140%200/l/v,c11570,t%2Bp


- Data da coleta: 19/3/2021;


- Possui metadados: sim, mas sem descrição das variáveis (https://metadados.ibge.gov.br/consulta/estatisticos/operacoes-estatisticas/CD).


### - Leitura do arquivo '.csv'.

Iniciamos com a visualização do arquivo '.csv' para verificarmos sua estrutura original e analisarmos as linhas que efetivamente deverão ser lidas.

In [82]:
nome_arquivo = 'tabela2030.csv'
enconding_arquivo = ler_csv(nome_arquivo) #Chama a função para a leitura do arquivo '.csv'

0
1 ['Tabela 2030 - Pessoas de 10 anos ou mais de idade, por classes de rendimento nominal mensal - Resultados Gerais da Amostra']
2 ['Variável - Pessoas de 10 anos ou mais de idade (Pessoas)']
3 ['Município', 'Ano', 'Classes de rendimento nominal mensal']
4 ['Município', 'Ano', 'Total', 'Até 1 salário mínimo', 'Mais de 1 a 2 salários mínimos', 'Mais de 2 a 3 salários mínimos', 'Mais de 3 a 5 salários mínimos', 'Mais de 5 a 10 salários mínimos', 'Mais de 10 a 20 salários mínimos', 'Mais de 20 salários mínimos', 'Sem rendimento']
5 ["Alta Floresta D'Oeste (RO)", '2010', '20434', '7202', '3525', '855', '732', '345', '76', '39', '7662']
6 ['Ariquemes (RO)', '2010', '74725', '19934', '17050', '5436', '4066', '2656', '604', '377', '24602']
7 ['Cabixi (RO)', '2010', '5291', '2033', '856', '241', '195', '77', '37', '10', '1841']
8 ['Cacoal (RO)', '2010', '66376', '18968', '13879', '4662', '3676', '2295', '677', '326', '21893']
9 ['Cerejeiras (RO)', '2010', '14454', '5565', '2518', '888', '776

285 ['Tailândia (PA)', '2010', '61104', '16698', '9066', '1838', '1178', '511', '160', '63', '31590']
286 ['Terra Alta (PA)', '2010', '8270', '3340', '703', '168', '75', '105', '9', '5', '3866']
287 ['Terra Santa (PA)', '2010', '13295', '5694', '1665', '262', '244', '133', '45', '28', '5225']
288 ['Tomé-Açu (PA)', '2010', '43610', '16963', '5962', '1444', '741', '391', '137', '93', '17879']
289 ['Tracuateua (PA)', '2010', '21342', '8326', '1422', '214', '217', '168', '-', '11', '10984']
290 ['Trairão (PA)', '2010', '13135', '4296', '1984', '503', '394', '192', '17', '9', '5739']
291 ['Tucumã (PA)', '2010', '27179', '9650', '5165', '1214', '968', '673', '274', '91', '9145']
292 ['Tucuruí (PA)', '2010', '77053', '22407', '13481', '4464', '3282', '2044', '566', '230', '30578']
293 ['Ulianópolis (PA)', '2010', '33585', '8267', '5434', '1229', '604', '429', '106', '62', '17455']
294 ['Uruará (PA)', '2010', '34993', '11867', '5057', '1389', '804', '524', '207', '35', '15111']
295 ['Vigia (PA

470 ['Arame (MA)', '2010', '23689', '8731', '1287', '268', '142', '165', '23', '16', '13057']
471 ['Arari (MA)', '2010', '23083', '9100', '2417', '782', '423', '284', '38', '23', '10017']
472 ['Axixá (MA)', '2010', '9034', '4308', '712', '142', '106', '58', '22', '-', '3686']
473 ['Bacabal (MA)', '2010', '80722', '38211', '9209', '2394', '1881', '1122', '376', '157', '27373']
474 ['Bacabeira (MA)', '2010', '11799', '4687', '1327', '237', '155', '91', '26', '14', '5262']
475 ['Bacuri (MA)', '2010', '13380', '6471', '1066', '120', '116', '101', '15', '25', '5466']
476 ['Bacurituba (MA)', '2010', '4262', '1740', '205', '43', '18', '6', '-', '-', '2250']
477 ['Balsas (MA)', '2010', '66366', '24093', '10984', '2917', '2350', '1957', '588', '336', '23141']
478 ['Barão de Grajaú (MA)', '2010', '14885', '5725', '1962', '431', '239', '158', '29', '12', '6329']
479 ['Barra do Corda (MA)', '2010', '66037', '28664', '5562', '1128', '1130', '685', '207', '123', '28538']
480 ['Barreirinhas (MA)', '2

581 ["Olho d'Água das Cunhãs (MA)", '2010', '15211', '7077', '1412', '213', '120', '106', '38', '10', '6235']
582 ['Olinda Nova do Maranhão (MA)', '2010', '10332', '4529', '656', '97', '72', '59', '15', '-', '4904']
583 ['Paço do Lumiar (MA)', '2010', '87273', '27064', '16683', '4348', '3204', '1785', '353', '89', '33745']
584 ['Palmeirândia (MA)', '2010', '15004', '6432', '1107', '113', '78', '32', '29', '-', '7214']
585 ['Paraibano (MA)', '2010', '16544', '6476', '1776', '448', '176', '144', '46', '20', '7457']
586 ['Parnarama (MA)', '2010', '26982', '11164', '2381', '233', '254', '120', '8', '-', '12822']
587 ['Passagem Franca (MA)', '2010', '13959', '6297', '1239', '197', '123', '66', '6', '13', '6019']
588 ['Pastos Bons (MA)', '2010', '14374', '6357', '1335', '191', '174', '94', '33', '-', '6191']
589 ['Paulino Neves (MA)', '2010', '10884', '3902', '840', '113', '46', '46', '14', '5', '5918']
590 ['Paulo Ramos (MA)', '2010', '15737', '7054', '1227', '248', '132', '127', '37', '17'

1380 ['Passagem (PB)', '2010', '1839', '898', '195', '33', '35', '5', '3', '-', '670']
1381 ['Patos (PB)', '2010', '84597', '36450', '11465', '3042', '2796', '1802', '615', '320', '28108']
1382 ['Paulista (PB)', '2010', '10067', '5711', '1051', '195', '115', '36', '13', '13', '2933']
1383 ['Pedra Branca (PB)', '2010', '3076', '1317', '248', '40', '11', '25', '-', '-', '1435']
1384 ['Pedra Lavrada (PB)', '2010', '6253', '3365', '600', '93', '54', '51', '5', '2', '2083']
1385 ['Pedras de Fogo (PB)', '2010', '22417', '10544', '2062', '493', '335', '123', '28', '27', '8805']
1386 ['Piancó (PB)', '2010', '13145', '6387', '1149', '350', '279', '108', '44', '4', '4824']
1387 ['Picuí (PB)', '2010', '15206', '7855', '1540', '317', '199', '138', '39', '24', '5094']
1388 ['Pilar (PB)', '2010', '9317', '4168', '787', '188', '77', '32', '5', '3', '4056']
1389 ['Pilões (PB)', '2010', '5648', '2455', '499', '98', '64', '43', '11', '9', '2469']
1390 ['Pilõezinhos (PB)', '2010', '4258', '2173', '334', 

...

2719 ['Matias Barbosa (MG)', '2010', '11721', '3580', '2502', '812', '534', '377', '117', '45', '3755']
2720 ['Matias Cardoso (MG)', '2010', '7956', '3527', '923', '148', '86', '47', '4', '28', '3193']
2721 ['Matipó (MG)', '2010', '14530', '6866', '2006', '404', '257', '196', '62', '55', '4683']
2722 ['Mato Verde (MG)', '2010', '10973', '6031', '1274', '266', '239', '86', '9', '5', '3061']
2723 ['Matozinhos (MG)', '2010', '29115', '8584', '7156', '2169', '1578', '544', '216', '30', '8839']
2724 ['Matutina (MG)', '2010', '3334', '1212', '705', '182', '107', '87', '16', '32', '993']
2725 ['Medeiros (MG)', '2010', '2966', '931', '669', '210', '123', '126', '38', '8', '860']
2726 ['Medina (MG)', '2010', '17589', '7429', '2014', '481', '415', '251', '54', '42', '6904']
2727 ['Mendes Pimentel (MG)', '2010', '5493', '2721', '580', '115', '107', '85', '11', '9', '1864']
2728 ['Mercês (MG)', '2010', '9041', '4461', '1162', '254', '240', '83', '43', '54', '2744']
2729 ['Mesquita (MG)', '2010', '

2854 ['Pompéu (MG)', '2010', '24598', '9144', '5262', '1456', '761', '442', '127', '192', '7215']
2855 ['Ponte Nova (MG)', '2010', '49856', '16770', '10343', '2994', '2431', '1682', '607', '223', '14805']
2856 ['Ponto Chique (MG)', '2010', '3249', '1475', '432', '68', '45', '4', '4', '-', '1220']
2857 ['Ponto dos Volantes (MG)', '2010', '9156', '3831', '977', '227', '137', '54', '4', '5', '3921']
2858 ['Porteirinha (MG)', '2010', '32243', '15994', '3408', '619', '494', '277', '66', '43', '11342']
2859 ['Porto Firme (MG)', '2010', '9158', '4273', '1246', '193', '118', '62', '13', '10', '3242']
2860 ['Poté (MG)', '2010', '13119', '6338', '1558', '232', '194', '94', '20', '9', '4674']
2861 ['Pouso Alegre (MG)', '2010', '113733', '24112', '30947', '10662', '8597', '5517', '1746', '780', '31374']
2862 ['Pouso Alto (MG)', '2010', '5524', '2180', '1089', '229', '209', '81', '31', '29', '1677']
2863 ['Prados (MG)', '2010', '7224', '3047', '1238', '309', '161', '122', '24', '27', '2296']
2864 [

2972 ['São João Batista do Glória (MG)', '2010', '6042', '1820', '1543', '486', '269', '151', '57', '28', '1688']
2973 ['São João da Lagoa (MG)', '2010', '3983', '1966', '401', '73', '41', '11', '14', '-', '1475']
2974 ['São João da Mata (MG)', '2010', '2435', '994', '550', '106', '102', '33', '10', '6', '635']
2975 ['São João da Ponte (MG)', '2010', '21371', '9504', '2287', '384', '326', '82', '6', '-', '8781']
2976 ['São João das Missões (MG)', '2010', '8869', '3356', '823', '128', '67', '5', '6', '-', '4484']
2977 ['São João del Rei (MG)', '2010', '74664', '23752', '15554', '5206', '4328', '2612', '1034', '305', '21874']
2978 ['São João do Manhuaçu (MG)', '2010', '8523', '3869', '1401', '326', '159', '60', '23', '31', '2654']
2979 ['São João do Manteninha (MG)', '2010', '4498', '2221', '566', '52', '78', '36', '9', '3', '1533']
2980 ['São João do Oriente (MG)', '2010', '6840', '3321', '900', '249', '200', '73', '13', '6', '2079']
2981 ['São João do Pacuí (MG)', '2010', '3388', '1717

4218 ['Ribeirão Claro (PR)', '2010', '9295', '3179', '2138', '467', '349', '232', '107', '21', '2801']
4219 ['Ribeirão do Pinhal (PR)', '2010', '11639', '4843', '2337', '521', '454', '291', '73', '39', '3081']
4220 ['Rio Azul (PR)', '2010', '11891', '4890', '2227', '784', '421', '203', '73', '16', '3277']
4221 ['Rio Bom (PR)', '2010', '2931', '1195', '641', '110', '83', '87', '8', '2', '803']
4222 ['Rio Bonito do Iguaçu (PR)', '2010', '11251', '4325', '2258', '472', '322', '122', '47', '17', '3688']
4223 ['Rio Branco do Ivaí (PR)', '2010', '3260', '1332', '583', '125', '59', '30', '2', '12', '1118']
4224 ['Rio Branco do Sul (PR)', '2010', '25480', '6816', '6454', '1952', '1195', '390', '172', '88', '8413']
4225 ['Rio Negro (PR)', '2010', '26576', '6639', '7312', '2291', '1551', '959', '285', '99', '7439']
4226 ['Rolândia (PR)', '2010', '50209', '9598', '16181', '4886', '3541', '1899', '539', '273', '13292']
4227 ['Roncador (PR)', '2010', '9859', '4368', '1569', '429', '333', '157', '64

4468 ['Major Gercino (SC)', '2010', '2858', '787', '843', '266', '147', '61', '27', '13', '714']
4469 ['Major Vieira (SC)', '2010', '6350', '2480', '1217', '249', '272', '118', '6', '8', '2000']
4470 ['Maracajá (SC)', '2010', '5481', '1131', '1841', '560', '379', '170', '37', '19', '1345']
4471 ['Maravilha (SC)', '2010', '19208', '5182', '5995', '1860', '1291', '1033', '285', '91', '3469']
4472 ['Marema (SC)', '2010', '1984', '586', '574', '208', '155', '70', '19', '6', '365']
4473 ['Massaranduba (SC)', '2010', '12828', '2225', '4506', '1934', '1306', '407', '117', '18', '2316']
4474 ['Matos Costa (SC)', '2010', '2339', '885', '465', '123', '76', '30', '8', '-', '752']
4475 ['Meleiro (SC)', '2010', '6167', '1690', '1764', '549', '431', '167', '67', '21', '1479']
4476 ['Mirim Doce (SC)', '2010', '2166', '559', '626', '168', '99', '38', '-', '-', '676']
4477 ['Modelo (SC)', '2010', '3531', '985', '1238', '363', '223', '151', '47', '21', '502']
4478 ['Mondaí (SC)', '2010', '9044', '2621',

Como se observa, acima, as primeiras linhas do arquivo são apenas informativas e não deverão ser carregadas (assim temos que configurar o parâmetro skiprows quando importarmos o csv).

Vemos também que, no fim do arquivo, há linhas que não nos interessam (totailizações, comentários, notas, etc). Assim, será necessário limitar a importação dos dados, configurando o parâmetro skipfooter (no caso).

Como utilizaremos o skipfooter, é necessário configurar o parâmetro engine='python'

In [83]:
IBGE_Censo_Rendimento=pd.read_csv(nome_arquivo, delimiter=',',engine='python', skiprows=3, skipfooter=16,encoding = enconding_arquivo)
IBGE_Censo_Rendimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Município                         5565 non-null   object
 1   Ano                               5565 non-null   int64 
 2   Total                             5565 non-null   int64 
 3   Até 1 salário mínimo              5565 non-null   int64 
 4   Mais de 1 a 2 salários mínimos    5565 non-null   int64 
 5   Mais de 2 a 3 salários mínimos    5565 non-null   int64 
 6   Mais de 3 a 5 salários mínimos    5565 non-null   int64 
 7   Mais de 5 a 10 salários mínimos   5565 non-null   object
 8   Mais de 10 a 20 salários mínimos  5565 non-null   object
 9   Mais de 20 salários mínimos       5565 non-null   object
 10  Sem rendimento                    5565 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 478.4+ KB


In [84]:
IBGE_Censo_Rendimento

,Município,Ano,Total,Até 1 salário mínimo,Mais de 1 a 2 salários mínimos,Mais de 2 a 3 salários mínimos,Mais de 3 a 5 salários mínimos,Mais de 5 a 10 salários mínimos,Mais de 10 a 20 salários mínimos,Mais de 20 salários mínimos,Sem rendimento
0,Alta Floresta D'Oeste (RO),2010,20434,7202,3525,855,732,345,76,39,7662
1,Ariquemes (RO),2010,74725,19934,17050,5436,4066,2656,604,377,24602
2,Cabixi (RO),2010,5291,2033,856,241,195,77,37,10,1841
3,Cacoal (RO),2010,66376,18968,13879,4662,3676,2295,677,326,21893
4,Cerejeiras (RO),2010,14454,5565,2518,888,776,284,81,51,4290
...,...,...,...,...,...,...,...,...,...,...,...
5560,Vianópolis (GO),2010,10754,3956,2356,626,496,387,91,32,2809
5561,Vicentinópolis (GO),2010,6192,1848,1526,521,239,186,52,25,1795
5562,Vila Boa (GO),2010,3807,1315,798,122,109,50,3,3,1409
5563,Vila Propício (GO),2010,4298,1307,758,197,101,35,13,3,1884


### - Promovendo transformações nos dados

- Obs: os dados disponíveis são unicamente os de 2010;
- Criar uma nova coluna, padronizada, chamada 'municipio-uf' que será a chave para joins/merges com outros dataframes;
- Atribuir nomes mais sucintos às variáveis;
- Criação de colunas adicionais;
- Excluindo colunas desnecessárias;
- Reordenar as colunas.

In [85]:
# Criando a nova coluna 'municipio-uf'.
IBGE_Censo_Rendimento['municipio-uf']=IBGE_Censo_Rendimento['Município'].str[:-5].str.upper()+'-'+IBGE_Censo_Rendimento['Município'].str[-3:-1].str.upper()

# Removendo caracteres especiais da coluna recém-criada.
IBGE_Censo_Rendimento['municipio-uf'] = IBGE_Censo_Rendimento['municipio-uf'].apply(remover_caracteres_especiais)

IBGE_Censo_Rendimento.head()

,Município,Ano,Total,Até 1 salário mínimo,Mais de 1 a 2 salários mínimos,Mais de 2 a 3 salários mínimos,Mais de 3 a 5 salários mínimos,Mais de 5 a 10 salários mínimos,Mais de 10 a 20 salários mínimos,Mais de 20 salários mínimos,Sem rendimento,municipio-uf
0,Alta Floresta D'Oeste (RO),2010,20434,7202,3525,855,732,345,76,39,7662,ALTA FLORESTA D'OESTE-RO
1,Ariquemes (RO),2010,74725,19934,17050,5436,4066,2656,604,377,24602,ARIQUEMES-RO
2,Cabixi (RO),2010,5291,2033,856,241,195,77,37,10,1841,CABIXI-RO
3,Cacoal (RO),2010,66376,18968,13879,4662,3676,2295,677,326,21893,CACOAL-RO
4,Cerejeiras (RO),2010,14454,5565,2518,888,776,284,81,51,4290,CEREJEIRAS-RO


In [86]:
print('Shape',IBGE_Censo_Rendimento.shape)
print('Valores únicos',pd.Series(IBGE_Censo_Rendimento['municipio-uf']).unique().shape)
# Veremos que não há duplicidades.

Shape (5565, 12)
Valores únicos (5565,)


In [87]:
#Vamos excluir as colunas em que os valores são fixos (e sem interesse): ano.
#Também excluiremos Município (que deu origem à coluna municipio-uf)
cols=['Ano','Município']
IBGE_Censo_Rendimento = IBGE_Censo_Rendimento.drop(cols,1)
IBGE_Censo_Rendimento.columns

Index(['Total', 'Até 1 salário mínimo', 'Mais de 1 a 2 salários mínimos',
       'Mais de 2 a 3 salários mínimos', 'Mais de 3 a 5 salários mínimos',
       'Mais de 5 a 10 salários mínimos', 'Mais de 10 a 20 salários mínimos',
       'Mais de 20 salários mínimos', 'Sem rendimento', 'municipio-uf'],
      dtype='object')

In [88]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Censo_Rendimento)

Shape do dataframe: (5565, 10)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [89]:
#Em seguida, vamos transformar as quantidades de pessoas em cada faixa em numérico.
cols = ['Total', 'Até 1 salário mínimo', 'Mais de 1 a 2 salários mínimos',
       'Mais de 2 a 3 salários mínimos', 'Mais de 3 a 5 salários mínimos',
       'Mais de 5 a 10 salários mínimos', 'Mais de 10 a 20 salários mínimos',
       'Mais de 20 salários mínimos', 'Sem rendimento']

for i in cols:
    IBGE_Censo_Rendimento[i]=pd.to_numeric(IBGE_Censo_Rendimento[i],errors='coerce')

IBGE_Censo_Rendimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Total                             5565 non-null   int64  
 1   Até 1 salário mínimo              5565 non-null   int64  
 2   Mais de 1 a 2 salários mínimos    5565 non-null   int64  
 3   Mais de 2 a 3 salários mínimos    5565 non-null   int64  
 4   Mais de 3 a 5 salários mínimos    5565 non-null   int64  
 5   Mais de 5 a 10 salários mínimos   5553 non-null   float64
 6   Mais de 10 a 20 salários mínimos  5175 non-null   float64
 7   Mais de 20 salários mínimos       4377 non-null   float64
 8   Sem rendimento                    5565 non-null   int64  
 9   municipio-uf                      5565 non-null   object 
dtypes: float64(3), int64(6), object(1)
memory usage: 434.9+ KB


In [90]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Censo_Rendimento)

Shape do dataframe: (5565, 10)
Colunas que contêm valores faltantes: ['Mais de 5 a 10 salários mínimos', 'Mais de 10 a 20 salários mínimos', 'Mais de 20 salários mínimos']
Número de linhas com valores faltantes: 1334 



['Mais de 5 a 10 salários mínimos',
 'Mais de 10 a 20 salários mínimos',
 'Mais de 20 salários mínimos']

In [91]:
## Vemos que para as colunas de rendimentros mais elevados (e apenas para elas) há valores NaN.

## Vamos substituir os NaN por zeros para não influenciar indevidamente a criação das novas colunas da próxima célula
## Somar número com NaN geraria erro.

IBGE_Censo_Rendimento = IBGE_Censo_Rendimento.fillna(0)
IBGE_Censo_Rendimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Total                             5565 non-null   int64  
 1   Até 1 salário mínimo              5565 non-null   int64  
 2   Mais de 1 a 2 salários mínimos    5565 non-null   int64  
 3   Mais de 2 a 3 salários mínimos    5565 non-null   int64  
 4   Mais de 3 a 5 salários mínimos    5565 non-null   int64  
 5   Mais de 5 a 10 salários mínimos   5565 non-null   float64
 6   Mais de 10 a 20 salários mínimos  5565 non-null   float64
 7   Mais de 20 salários mínimos       5565 non-null   float64
 8   Sem rendimento                    5565 non-null   int64  
 9   municipio-uf                      5565 non-null   object 
dtypes: float64(3), int64(6), object(1)
memory usage: 434.9+ KB


In [92]:
# Temos, então, 5565 municípios com informações completas.

#Vamos, agora, transformar a informação como um percentual do total.

IBGE_Censo_Rendimento['07_Ate_1SM_%'] = IBGE_Censo_Rendimento['Até 1 salário mínimo']/IBGE_Censo_Rendimento['Total']*100
IBGE_Censo_Rendimento['07_Ate_2SM_%'] = IBGE_Censo_Rendimento['Mais de 1 a 2 salários mínimos']/IBGE_Censo_Rendimento['Total']*100
IBGE_Censo_Rendimento['07_Ate_3SM_%'] = IBGE_Censo_Rendimento['Mais de 2 a 3 salários mínimos']/IBGE_Censo_Rendimento['Total']*100
IBGE_Censo_Rendimento['07_Ate_5SM_%'] = IBGE_Censo_Rendimento['Mais de 3 a 5 salários mínimos']/IBGE_Censo_Rendimento['Total']*100
IBGE_Censo_Rendimento['07_Ate_10SM_%'] = IBGE_Censo_Rendimento['Mais de 5 a 10 salários mínimos']/IBGE_Censo_Rendimento['Total']*100
IBGE_Censo_Rendimento['07_Ate_20SM_%'] = IBGE_Censo_Rendimento['Mais de 10 a 20 salários mínimos']/IBGE_Censo_Rendimento['Total']*100
IBGE_Censo_Rendimento['07_20SM+_%'] = IBGE_Censo_Rendimento['Mais de 20 salários mínimos']/IBGE_Censo_Rendimento['Total']*100
IBGE_Censo_Rendimento['07_Sem_Rendimento_%'] = IBGE_Censo_Rendimento['Sem rendimento']/IBGE_Censo_Rendimento['Total']*100

#Podemos agora apagar as colunas originais.
cols=[ 'Até 1 salário mínimo', 'Mais de 1 a 2 salários mínimos',
       'Mais de 2 a 3 salários mínimos', 'Mais de 3 a 5 salários mínimos',
       'Mais de 5 a 10 salários mínimos', 'Mais de 10 a 20 salários mínimos',
       'Mais de 20 salários mínimos', 'Sem rendimento']

IBGE_Censo_Rendimento = IBGE_Censo_Rendimento.drop(cols,1)

IBGE_Censo_Rendimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Total                5565 non-null   int64  
 1   municipio-uf         5565 non-null   object 
 2   07_Ate_1SM_%         5565 non-null   float64
 3   07_Ate_2SM_%         5565 non-null   float64
 4   07_Ate_3SM_%         5565 non-null   float64
 5   07_Ate_5SM_%         5565 non-null   float64
 6   07_Ate_10SM_%        5565 non-null   float64
 7   07_Ate_20SM_%        5565 non-null   float64
 8   07_20SM+_%           5565 non-null   float64
 9   07_Sem_Rendimento_%  5565 non-null   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 434.9+ KB


In [93]:
# Chama a função para identificar se há valores faltantes ou NaN.
identificar_faltantes(IBGE_Censo_Rendimento)

Shape do dataframe: (5565, 10)
Colunas que contêm valores faltantes: []
Número de linhas com valores faltantes: 0 



[]

In [94]:
# Vamos renomar a coluna Total

IBGE_Censo_Rendimento = IBGE_Censo_Rendimento.rename({'Total': '07_total_rendimentos'}, axis = 1)

# ...e agora reorndeá-las.

cols = ['municipio-uf','07_total_rendimentos', '07_Ate_1SM_%', '07_Ate_2SM_%', '07_Ate_3SM_%',
       '07_Ate_5SM_%', '07_Ate_10SM_%', '07_Ate_20SM_%', '07_20SM+_%',
       '07_Sem_Rendimento_%']

IBGE_Censo_Rendimento = IBGE_Censo_Rendimento[cols]

print(IBGE_Censo_Rendimento.shape)
IBGE_Censo_Rendimento.info()

(5565, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   municipio-uf          5565 non-null   object 
 1   07_total_rendimentos  5565 non-null   int64  
 2   07_Ate_1SM_%          5565 non-null   float64
 3   07_Ate_2SM_%          5565 non-null   float64
 4   07_Ate_3SM_%          5565 non-null   float64
 5   07_Ate_5SM_%          5565 non-null   float64
 6   07_Ate_10SM_%         5565 non-null   float64
 7   07_Ate_20SM_%         5565 non-null   float64
 8   07_20SM+_%            5565 non-null   float64
 9   07_Sem_Rendimento_%   5565 non-null   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 434.9+ KB


In [95]:
IBGE_Censo_Rendimento.head()

,municipio-uf,07_total_rendimentos,07_Ate_1SM_%,07_Ate_2SM_%,07_Ate_3SM_%,07_Ate_5SM_%,07_Ate_10SM_%,07_Ate_20SM_%,07_20SM+_%,07_Sem_Rendimento_%
0,ALTA FLORESTA D'OESTE-RO,20434,35.245180,17.250661,4.184203,3.582265,1.688363,0.371929,0.190858,37.496330
1,ARIQUEMES-RO,74725,26.676480,22.816996,7.274674,5.441285,3.554366,0.808297,0.504517,32.923386
2,CABIXI-RO,5291,38.423738,16.178416,4.554905,3.685504,1.455301,0.699301,0.189000,34.794935
3,CACOAL-RO,66376,28.576594,20.909666,7.023623,5.538146,3.457575,1.019947,0.491141,32.983307
4,CEREJEIRAS-RO,14454,38.501453,17.420783,6.143628,5.368756,1.964854,0.560399,0.352844,29.680365


### - Cria '.csv' com o resultado das transformações na base original.

In [96]:
IBGE_Censo_Rendimento.to_csv('ntbk 01_07 - ibge censo rendimentos.csv',index=False)

# 3. Resumo das bases de dados criadas

In [97]:
#Ntbk 01_01 - RFB Arrecadacao 2018.csv
arrecadacao_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   municipio-uf         5576 non-null   object 
 1   01_arrecadacao_2018  5576 non-null   float64
dtypes: float64(1), object(1)
memory usage: 87.2+ KB


In [98]:
#Ntbk 01_02 - IBGE Empresas 2018.csv
IBGE_Empresas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   municipio-uf                      5570 non-null   object 
 1   02_unidades                       5570 non-null   int64  
 2   02_unidades_atuantes_%            5570 non-null   float64
 3   02_ocupados_total                 5570 non-null   int64  
 4   02_ocupados_assalariados_%        5570 non-null   float64
 5   02_ocupados_assalariados_medio    5570 non-null   float64
 6   02_salario_medio_mensal_em_SM     5570 non-null   float64
 7   02_salario_medio_mensal_em_Reais  5570 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 391.6+ KB


In [99]:
#Ntbk 01_03 - IBGE PIB 2018.csv
IBGE_PIB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 44545 to 50114
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   municipio-uf                       5570 non-null   object 
 1   03_nome_uf                         5570 non-null   object 
 2   03_uf                              5570 non-null   object 
 3   03_cod_municipio                   5570 non-null   object 
 4   03_grande_regiao                   5570 non-null   object 
 5   03_mesorregiao                     5570 non-null   object 
 6   03_microregiao                     5570 non-null   object 
 7   03_regiao_imediata                 5570 non-null   object 
 8   03_regiao_intermediaria            5570 non-null   object 
 9   03_vlr_adic_bruto_total            5570 non-null   float64
 10  03_vlr_adic_bruto_agropecuaria_%   5570 non-null   float64
 11  03_vlr_adic_bruto_industria_%      5570 non-null   

In [100]:
#Ntbk 01_04 - IBGE População Residente 2018.csv
IBGE_Residentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 2 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   municipio-uf                 5570 non-null   object
 1   04_populacao_residente_2018  5570 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [101]:
#Ntbk 01_05 - IBGE Censo 2010 Idades.csv
IBGE_Censo_Idade.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5565
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   municipio-uf         5565 non-null   object 
 1   05_total_residentes  5565 non-null   float64
 2   05_0-19_anos_%       5565 non-null   float64
 3   05_20-39_anos_%      5565 non-null   float64
 4   05_40-59_anos_%      5565 non-null   float64
 5   05_60+_anos_%        5565 non-null   float64
dtypes: float64(5), object(1)
memory usage: 304.3+ KB


In [102]:
#Ntbk 01_06 - IBGE Censo Instrucao.csv
IBGE_Censo_Instrucao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   municipio-uf                                5565 non-null   object 
 1   06_total_instrução                          5565 non-null   int64  
 2   06_Sem_Instrução_e_Fundamental_Incompleto%  5565 non-null   float64
 3   06_Medio_Incompleto_%                       5565 non-null   float64
 4   06_Superior_Incompleto_%                    5565 non-null   float64
 5   06_Superior_Completo_%                      5565 non-null   float64
 6   06_Indeterminado_%                          5565 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 304.5+ KB


In [103]:
#Ntbk 01_07 - IBGE Censo Rendimentos.csv
IBGE_Censo_Rendimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   municipio-uf          5565 non-null   object 
 1   07_total_rendimentos  5565 non-null   int64  
 2   07_Ate_1SM_%          5565 non-null   float64
 3   07_Ate_2SM_%          5565 non-null   float64
 4   07_Ate_3SM_%          5565 non-null   float64
 5   07_Ate_5SM_%          5565 non-null   float64
 6   07_Ate_10SM_%         5565 non-null   float64
 7   07_Ate_20SM_%         5565 non-null   float64
 8   07_20SM+_%            5565 non-null   float64
 9   07_Sem_Rendimento_%   5565 non-null   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 434.9+ KB
